# ShallowConvNet

This notebook provides a modified reimplementation of the popular CNN-based EEG classifier: ShallowConvNet by [Schirrmeister et al](https://doi.org/10.1002/hbm.23730).
It is a less deep variant of the ShallowConvNet model explored in the previous paper notebook, notebook `5-deepconvnet.ipynb`.
Most modifications made include parameter changes to make it better suited for our data.
The knowledge and utilities obtained from experimental notebooks six and seven are used throughout this notebook.

This notebook works in an offline fashion and uses epochs with a length of 3 seconds.
This epoch starts 1 second before the visual queue was given, includes the 1 second the visual queue was shown and ends 1 second after the visual queue was hidden, totalling 3 seconds.
No baseline correction was performed and the raw EEG data was used.
The effective training and testing are done in a half-second window, starting 0.1 seconds after the start of the visual queue.
A window of 0.5 seconds was chosen as it is a common size for sliding window approaches in online systems.
Some alternatives to this setup were also considered, including one alternative performed for all experiment which uses a long, 1.5 seconds, window.


Instructions on where to get the data are available on [the GitHub repository of the BCI master thesis project](https://www.github.com/pikawika/bci-master-thesis). These instructions are under `bci-master-thesis/code/data/CLA/README.md`. We will use the utility file `bci-master-thesis/code/utils/CLA_dataset.py` to work with this data. The data was stored as FIF files, which are included in [the GitHub repository of the BCI master thesis project](https://www.github.com/pikawika/bci-master-thesis).

<hr><hr>

## Table of Contents

- Checking requirements
   - Correct Anaconda environment
   - Correct module access
   - Correct file access
   - Checking TensorFlow support
- Same subject, same session
   - Results
   - Longer window length 
- Same subject, new session
   - Results
   - Longer window length 
- New subject
   - Results
   - Longer window length

<hr><hr>

## Checking requirements

### Correct Anaconda environment

The `bci-master-thesis` Anaconda environment should be active to ensure proper support. Installation instructions are available on [the GitHub repository of the BCI master thesis project](https://www.github.com/pikawika/bci-master-thesis).

In [1]:
####################################################
# CHECKING FOR RIGHT ANACONDA ENVIRONMENT
####################################################

import os
from platform import python_version
from pathlib import Path
from copy import copy

print(f"Active environment: {os.environ['CONDA_DEFAULT_ENV']}")
print(f"Correct environment: {os.environ['CONDA_DEFAULT_ENV'] == 'bci-master-thesis'}")
print(f"\nPython version: {python_version()}")
print(f"Correct Python version: {python_version() == '3.8.10'}")

Active environment: bci-master-thesis
Correct environment: True

Python version: 3.8.10
Correct Python version: True


<hr>

### Correct module access

The following code block will load in all required modules.

In [2]:
####################################################
# LOADING MODULES
####################################################

# allow reloading of libraries
import importlib

# Load util function file
import sys
sys.path.append('../utils')
import CLA_dataset
import TF_tools
importlib.reload(CLA_dataset)
importlib.reload(TF_tools)

# IO functions
from IPython.utils import io

# Set logging level for MNE before loading MNE
os.environ['MNE_LOGGING_LEVEL'] = 'WARNING'

# Modules tailored for EEG data
import mne; print(f"MNE version (1.0.2 recommended): {mne.__version__}")

# ShallowConvNet model
import EEGModels
from EEGModels import ShallowConvNet

# Data manipulation modules
import numpy as np; print(f"Numpy version (1.21.5 recommended): {np.__version__}")
import pandas as pd; print(f"Pandas version (1.4.1 recommended): {pd.__version__}")
import copy

# ML libraries
import sklearn;  print(f"Scikit-learn version (1.0.2 recommended): {sklearn.__version__}")
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score
from sklearn.preprocessing import OneHotEncoder

# Deep Learning libraries
import tensorflow as tf;  print(f"TensorFlow version (2.8.0 recommended): {tf.__version__}")

import keras; print(f"Keras version (2.8.0 recommended): {keras.__version__}")
from keras.callbacks import ModelCheckpoint

# Storing files
import pickle;  print(f"Pickle version (4.0 recommended): {pickle.format_version}")

# Plotting
import matplotlib; print(f"Matplotlib version (3.5.1 recommended): {matplotlib.__version__}")
import matplotlib.pyplot as plt

MNE version (1.0.2 recommended): 1.0.2
Numpy version (1.21.5 recommended): 1.21.5
Pandas version (1.4.1 recommended): 1.4.1
Scikit-learn version (1.0.2 recommended): 1.0.2
TensorFlow version (2.8.0 recommended): 2.8.0
Keras version (2.8.0 recommended): 2.8.0
Pickle version (4.0 recommended): 4.0
Matplotlib version (3.5.1 recommended): 3.5.1


<hr>

### Correct file access

As mentioned, this notebook uses a database provided by [Kaya et al](https://doi.org/10.1038/sdata.2018.211). The CLA dataset in particular. Instructions on where to get the data are available on [the GitHub repository of the BCI master thesis project](https://www.github.com/pikawika/bci-master-thesis). These instructions are under `bci-master-thesis/code/data/CLA/README.md`. The following code block checks if all required files are available.

In [3]:
####################################################
# CHECKING FILE ACCESS
####################################################

# Use util to determine if we have access
print("Full Matlab CLA file access: " + str(CLA_dataset.check_matlab_files_availability()))
print("Full MNE CLA file access: " + str(CLA_dataset.check_mne_files_availability()))

Full Matlab CLA file access: True
Full MNE CLA file access: True


<hr>

### Checking TensorFlow support

If you want to use TensorFlow with GPU acceleration, the below codeblock can help you gather insight.

To launch the tensorboard use the following command in the `paper-notebooks` folder, be sure to have the right environments active:
- Windows: `tensorboard --logdir=./logs/`
- MacOS: `tensorboard --logdir='./logs/'`

In [4]:
TF_tools.check_tf_cpu_gpu_presence()

There are 1 CPUs available under the names:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


There are 1 GPUs available under the names:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


<hr><hr>

## Same subject, same session

As discussed in the master's thesis, training and testing a classification system can happen using multiple strategies.
A classifier may be trained on a singular subject, using a singular session and testing on that same session.
This is an over-optimistic testing scenario and has a great risk of overfitting with poor generalisation to new sessions or new subjects but can be an okay baseline test to see if *at least something* can be learned.
Just like we did for the previous approaches, we do this for the ShallowConvNet model as well.


This experiment works as follows:
   - We use participants with at least three recordings
      - Participants: B, C, E
      - NOTE: participant F has three files provided but one of those files has only three MI classes rather than three, hence it is not considered here
   - We use the last recorded session of each of these participants, thus the one where the participant has the most experience
   - We get epochs of 3 seconds, which includes one second before and after the visual queue
      - We use only a half a second window taking into account the online system will use sliding windows.
      - This window starts at 0.1 seconds after then visual queue and ends at 0.6 seconds after the visual queue
   - We split the data in a train/test dataset with 20% test data balanced over all MI classes
   - We use the parameters for ShallowConvNet as found in the experimental notebook `6-DL-based-classification.ipynb`:
      - We input the raw data, which is not baseline corrected, but multiply it by 1000000 to combat the scaling that MNE does per default.
         - This means that we only use the data from the window, so it is easier adoptable to a live setting.
      - We used the modified ShallowConvNet model provided through the `EEGModels.py` util file with the following settings:
         - nb_classes = 3 (int, number of classes to classify)
         - Chans = 21 (number of channels in the EEG data)
         - Samples = 100 (number of time points in the EEG data - default: 128, paper: 250)
         - dropoutRate = 0.8 (dropout fraction - default: 0.5)
         - conv_filters = 10 (Conv2D kernel size - default: 13, paper: 25)
         - strides = 6 (Stride size for average pooling layer - default: 7, paper: 15)
         - pool_size = 30 (Pool size for average pooling layer - default: 35, paper: 75)
      - We trained for 2500 epochs, saving the best model based on best validation accuracy and validation loss (0.25 validation split - equal size as true test split)
   - We record the validation accuracy and loss over time for monitoring the training and test on the seperate test set

In [9]:
####################################################
# TRAINING SHALLOWCONVNET FOR EACH SUBJECT AND SESSION
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

do_experiment = True # Long experiment disabled per default

# Function to create the TensorFlow Keras model as clone_model doesn't work with custom objects 
def get_keras_shallowcn_model():
        return ShallowConvNet(
                nb_classes = 3, # int, number of classes to classify. 
                Chans = 21, # number of channels in the EEG data. 
                Samples = 100, # number of time points in the EEG data. (default: 128, paper: 250)
                dropoutRate = 0.8, # dropout fraction. (default: 0.5)
                conv_filters = 10, # Conv2D kernel size (default: 13, paper: 25)
                strides = 6, # Stride size for average pooling layer (default: 7, paper: 15)
                pool_size = 30 # Pool size for average pooling layer (default: 35, paper: 75)
                )

if do_experiment:
        # Loop over all subjects and perform the grid search for finding the best parameters
        for subject_id in subject_ids_to_test:
                print("")
                print("####################################################")
                print(f"# TRAINING FOR SUBJECT {subject_id}")
                print("####################################################")
                print("")
                ###################### PREPARE DATA ######################
                
                with io.capture_output():
                        # Get MNE raw object for latest recording of that subject
                        mne_raw = CLA_dataset.get_last_raw_mne_data_for_subject(subject_id= subject_id)
                        
                        # Get epochs for that MNE raw
                        mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                                             start_offset= start_offset,
                                                                             end_offset= end_offset,
                                                                             baseline= baseline)
                        
                        # Only keep epochs from the MI tasks
                        mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']
                        
                        # Load epochs into memory
                        mne_epochs.load_data()
                
                # Get the labels
                labels = mne_epochs.events[:, -1]
                
                # Convert the labels to OHE labels as needed for Keras
                labels = labels.reshape(-1, 1)
                ohe = OneHotEncoder()
                labels = ohe.fit_transform(labels).toarray()
                
                # Get a half second window
                mne_epochs_data = mne_epochs.get_data(tmin= 0.1, tmax= 0.6)
                
                # Fix scaling sensitivity as MNE stores as data * 10e-6
                mne_epochs_data = mne_epochs_data * 1000000
                
                # Create a test and train split
                X_train, X_test, y_train, y_test = train_test_split(mne_epochs_data,
                                                                    labels,
                                                                    test_size = 0.2,
                                                                    shuffle= True,
                                                                    stratify= labels,                                                    
                                                                    random_state= 1998)
                
                # Store the train and test data so the best model can be retrained later
                with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/testdata-x.pickle", 'wb') as file:
                        pickle.dump(X_test, file)
                with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/testdata-y.pickle", 'wb') as file:
                        pickle.dump(y_test, file)
                with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/traindata-x.pickle", 'wb') as file:
                        pickle.dump(X_train, file)
                with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/traindata-y.pickle", 'wb') as file:
                        pickle.dump(y_train, file)
                        
                # Store the OHE encoder to enable same conversion later
                with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/ohe-encoder.pickle", 'wb') as file:
                        pickle.dump(ohe, file)
                        
                print(f"Shape of all data (epochs, channels, samples): {np.shape(mne_epochs_data)}")
                print(f"Shape of train data (epochs, channels, samples): {np.shape(X_train)}")
                print(f"Shape of test data (epochs, channels, samples): {np.shape(X_test)}")
                
                
                ###################### PREPARE MODEL ######################
                
                # Names for model
                best_base_model_filename = f"saved_variables/6/samesubject_samesession/subject{subject_id}/trained_model" 
                tensorboard_name = f"paper-notebook6_deepconvnet_singlesession_subject{subject_id}" # log name for tensorboard
                
                # Create copy of the model that needs training
                trained_model = get_keras_shallowcn_model()
                
                # Compile the model so it can be fitted (loss and optimizer from ShallowConvNet paper)
                trained_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
                
                # Train model with GPU
                # NOTE: change GPU to CPU if nog GPU present
                with tf.device('/gpu:0'):
                        history = trained_model.fit(
                                x= X_train,
                                y= y_train,
                                batch_size= 128, # Default: 32
                                epochs= 2500, # Default: 500
                                verbose= 1, # 0 = silent, 1 = progress bar, 2 = one line per epoch
                                callbacks= [TF_tools.tensorboard_callback(log_name= tensorboard_name),
                                            TF_tools.lowest_loss_model_save_callback(filepath= best_base_model_filename),
                                            TF_tools.highest_accuracy_model_save_callback(filepath= best_base_model_filename)],
                                validation_split= 0.25,
                                shuffle= True,
                                sample_weight= None, # Can be interesting due to time series
                                use_multiprocessing=True, # Done for faster speed
                                workers= 4 # Done for faster speed
                                )
                        
                # Store the fitting history
                with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/fitting_history.pickle", 'wb') as file:
                        pickle.dump(history.history, file)
                
                # Delete vars after singular experiment
                del mne_raw
                del mne_epochs
                del mne_epochs_data
                del trained_model
                del best_base_model_filename
                del tensorboard_name
                del labels
                del file
                del history
                del X_train
                del X_test
                del ohe
                del y_train
                del y_test
    
        # Delete vars after all experiments
        del subject_id
        
# Del global vars
del subject_ids_to_test
del baseline
del do_experiment
del end_offset
del start_offset


####################################################
# TRAINING FOR SUBJECT B
####################################################

Shape of all data (epochs, channels, samples): (960, 21, 100)
Shape of train data (epochs, channels, samples): (768, 21, 100)
Shape of test data (epochs, channels, samples): (192, 21, 100)
Epoch 1/2500
5/5 [==============================] - ETA: 0s - loss: 1.6331 - accuracy: 0.3351
Epoch 1: val_loss improved from inf to 1.10072, saving model to saved_variables/6/samesubject_samesession/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 1: val_accuracy improved from -inf to 0.38542, saving model to saved_variables/6/samesubject_samesession/subjectB\trained_model_highest_acc_model.hdf5
5/5 [==============================] - 1s 87ms/step - loss: 1.6331 - accuracy: 0.3351 - val_loss: 1.1007 - val_accuracy: 0.3854
Epoch 2/2500
5/5 [==============================] - ETA: 0s - loss: 1.2709 - accuracy: 0.3767
Epoch 2: val_loss improved from 1.10072 to 1.04179, 

Epoch 17/2500
1/5 [=====>........................] - ETA: 0s - loss: 0.6955 - accuracy: 0.7266
Epoch 17: val_loss did not improve from 0.88261

Epoch 17: val_accuracy did not improve from 0.62500
5/5 [==============================] - 0s 21ms/step - loss: 0.7300 - accuracy: 0.6771 - val_loss: 0.8898 - val_accuracy: 0.6146
Epoch 18/2500
1/5 [=====>........................] - ETA: 0s - loss: 0.8025 - accuracy: 0.6484
Epoch 18: val_loss did not improve from 0.88261

Epoch 18: val_accuracy did not improve from 0.62500
5/5 [==============================] - 0s 20ms/step - loss: 0.6946 - accuracy: 0.7118 - val_loss: 0.9012 - val_accuracy: 0.5885
Epoch 19/2500
1/5 [=====>........................] - ETA: 0s - loss: 0.6329 - accuracy: 0.7734
Epoch 19: val_loss did not improve from 0.88261

Epoch 19: val_accuracy did not improve from 0.62500
5/5 [==============================] - 0s 25ms/step - loss: 0.7011 - accuracy: 0.6962 - val_loss: 0.8926 - val_accuracy: 0.5781
Epoch 20/2500
4/5 [=========

5/5 [==============================] - 0s 26ms/step - loss: 0.5005 - accuracy: 0.8073 - val_loss: 0.8542 - val_accuracy: 0.6354
Epoch 40/2500
5/5 [==============================] - ETA: 0s - loss: 0.4589 - accuracy: 0.8351
Epoch 40: val_loss did not improve from 0.81453

Epoch 40: val_accuracy did not improve from 0.65625
5/5 [==============================] - 0s 59ms/step - loss: 0.4589 - accuracy: 0.8351 - val_loss: 0.8309 - val_accuracy: 0.6458
Epoch 41/2500
5/5 [==============================] - ETA: 0s - loss: 0.4771 - accuracy: 0.8177
Epoch 41: val_loss did not improve from 0.81453

Epoch 41: val_accuracy improved from 0.65625 to 0.67188, saving model to saved_variables/6/samesubject_samesession/subjectB\trained_model_highest_acc_model.hdf5
5/5 [==============================] - 0s 35ms/step - loss: 0.4771 - accuracy: 0.8177 - val_loss: 0.8291 - val_accuracy: 0.6719
Epoch 42/2500
5/5 [==============================] - ETA: 0s - loss: 0.4667 - accuracy: 0.8142
Epoch 42: val_loss d

Epoch 63/2500
5/5 [==============================] - ETA: 0s - loss: 0.3643 - accuracy: 0.8854
Epoch 63: val_loss did not improve from 0.78272

Epoch 63: val_accuracy did not improve from 0.69271
5/5 [==============================] - 0s 23ms/step - loss: 0.3643 - accuracy: 0.8854 - val_loss: 0.8291 - val_accuracy: 0.6458
Epoch 64/2500
1/5 [=====>........................] - ETA: 0s - loss: 0.3564 - accuracy: 0.8984
Epoch 64: val_loss did not improve from 0.78272

Epoch 64: val_accuracy did not improve from 0.69271
5/5 [==============================] - 0s 22ms/step - loss: 0.3672 - accuracy: 0.8750 - val_loss: 0.8127 - val_accuracy: 0.6667
Epoch 65/2500
1/5 [=====>........................] - ETA: 0s - loss: 0.3121 - accuracy: 0.8906
Epoch 65: val_loss did not improve from 0.78272

Epoch 65: val_accuracy did not improve from 0.69271
5/5 [==============================] - 0s 21ms/step - loss: 0.3298 - accuracy: 0.8889 - val_loss: 0.8068 - val_accuracy: 0.6615
Epoch 66/2500
1/5 [=====>...

Epoch 88/2500
1/5 [=====>........................] - ETA: 0s - loss: 0.2547 - accuracy: 0.9297
Epoch 88: val_loss did not improve from 0.77917

Epoch 88: val_accuracy did not improve from 0.69271
5/5 [==============================] - 0s 22ms/step - loss: 0.2848 - accuracy: 0.9184 - val_loss: 0.8289 - val_accuracy: 0.6406
Epoch 89/2500
1/5 [=====>........................] - ETA: 0s - loss: 0.2492 - accuracy: 0.9453
Epoch 89: val_loss did not improve from 0.77917

Epoch 89: val_accuracy did not improve from 0.69271
5/5 [==============================] - 0s 21ms/step - loss: 0.3132 - accuracy: 0.9115 - val_loss: 0.8119 - val_accuracy: 0.6458
Epoch 90/2500
1/5 [=====>........................] - ETA: 0s - loss: 0.3024 - accuracy: 0.9141
Epoch 90: val_loss did not improve from 0.77917

Epoch 90: val_accuracy did not improve from 0.69271
5/5 [==============================] - 0s 21ms/step - loss: 0.2942 - accuracy: 0.9115 - val_loss: 0.7978 - val_accuracy: 0.6510
Epoch 91/2500
1/5 [=====>...

...

1/5 [=====>........................] - ETA: 0s - loss: 0.1672 - accuracy: 0.9688
Epoch 380: val_loss did not improve from 0.68277

Epoch 380: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 22ms/step - loss: 0.1584 - accuracy: 0.9757 - val_loss: 0.6975 - val_accuracy: 0.7031
Epoch 381/2500
5/5 [==============================] - ETA: 0s - loss: 0.1806 - accuracy: 0.9583
Epoch 381: val_loss did not improve from 0.68277

Epoch 381: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 24ms/step - loss: 0.1806 - accuracy: 0.9583 - val_loss: 0.7168 - val_accuracy: 0.7083
Epoch 382/2500
5/5 [==============================] - ETA: 0s - loss: 0.1707 - accuracy: 0.9740
Epoch 382: val_loss did not improve from 0.68277

Epoch 382: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 23ms/step - loss: 0.1707 - accuracy: 0.9740 - val_loss: 0.7212 - val_accuracy: 0.6875
Epoch 383/2500
5/5 [==============


Epoch 404: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 30ms/step - loss: 0.1608 - accuracy: 0.9792 - val_loss: 0.6995 - val_accuracy: 0.7135
Epoch 405/2500
5/5 [==============================] - ETA: 0s - loss: 0.1439 - accuracy: 0.9826
Epoch 405: val_loss did not improve from 0.67873

Epoch 405: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 28ms/step - loss: 0.1439 - accuracy: 0.9826 - val_loss: 0.6940 - val_accuracy: 0.7135
Epoch 406/2500
4/5 [=======================>......] - ETA: 0s - loss: 0.1688 - accuracy: 0.9668
Epoch 406: val_loss did not improve from 0.67873

Epoch 406: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 42ms/step - loss: 0.1617 - accuracy: 0.9688 - val_loss: 0.6937 - val_accuracy: 0.7135
Epoch 407/2500
5/5 [==============================] - ETA: 0s - loss: 0.1532 - accuracy: 0.9705
Epoch 407: val_loss did not improve from 0.67873

Epoch 407: val_acc

5/5 [==============================] - 0s 24ms/step - loss: 0.1526 - accuracy: 0.9792 - val_loss: 0.7072 - val_accuracy: 0.7031
Epoch 430/2500
5/5 [==============================] - ETA: 0s - loss: 0.1780 - accuracy: 0.9722
Epoch 430: val_loss did not improve from 0.67873

Epoch 430: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 35ms/step - loss: 0.1780 - accuracy: 0.9722 - val_loss: 0.7330 - val_accuracy: 0.6875
Epoch 431/2500
4/5 [=======================>......] - ETA: 0s - loss: 0.1772 - accuracy: 0.9688
Epoch 431: val_loss did not improve from 0.67873

Epoch 431: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 27ms/step - loss: 0.1840 - accuracy: 0.9653 - val_loss: 0.7393 - val_accuracy: 0.6875
Epoch 432/2500
5/5 [==============================] - ETA: 0s - loss: 0.1638 - accuracy: 0.9688
Epoch 432: val_loss did not improve from 0.67873

Epoch 432: val_accuracy did not improve from 0.73438
5/5 [==============

Epoch 455/2500
5/5 [==============================] - ETA: 0s - loss: 0.1527 - accuracy: 0.9757
Epoch 455: val_loss did not improve from 0.67873

Epoch 455: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 23ms/step - loss: 0.1527 - accuracy: 0.9757 - val_loss: 0.7177 - val_accuracy: 0.7031
Epoch 456/2500
5/5 [==============================] - ETA: 0s - loss: 0.1558 - accuracy: 0.9792
Epoch 456: val_loss did not improve from 0.67873

Epoch 456: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 26ms/step - loss: 0.1558 - accuracy: 0.9792 - val_loss: 0.7046 - val_accuracy: 0.6927
Epoch 457/2500
5/5 [==============================] - ETA: 0s - loss: 0.1818 - accuracy: 0.9635
Epoch 457: val_loss did not improve from 0.67873

Epoch 457: val_accuracy did not improve from 0.73438
5/5 [==============================] - 0s 23ms/step - loss: 0.1818 - accuracy: 0.9635 - val_loss: 0.7041 - val_accuracy: 0.7083
Epoch 458/2500
4/5 

KeyboardInterrupt: 

#### Results 

| **Subject** | **ShallowConvNet: best validation accuracy** | **ShallowConvNet: best validation loss** | **ShallowConvNet: test split accuracy (best acc model)** | **ShallowConvNet: test split accuracy (best loss model)** |
|-------------|-------------------------------------------|---------------------------------------|-------------------------------------------------------|--------------------------------------------------------|
| B           | 0.8073 @ epoch 2243                       | 0.6086 @ epoch 197                    | 0.8073                                                | 0.7604                                                 |
| C           | 0.8906 @ epoch 1143                       | 0.4337 @ epoch 41                     | 0.8802                                                | 0.8854                                                 |
| E           | 0.9215 @ epoch 221                        | 0.2354 @ epoch 458                    | 0.8743                                                | 0.8848                                                 ||

The training plots are given below.
B is dark blue, C is light blue and E is light green.
Convergence is already reached after a 100-200 epochs.

![Accuracy plot](figures/6/samesubject_samesession/accuracy.png)
![Loss plot](figures/6/samesubject_samesession/loss.png)

In [ ]:
####################################################
# RESULTS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings

# Loop over all found results
for subject_id in subject_ids_to_test:
    print()
    print("####################################################")
    print(f"# RESULTS FOR SUBJECT {subject_id}")
    print("####################################################")
    print()
    
    ################### load data ###################
    # Names for model
    best_base_model_filename = f"saved_variables/6/samesubject_samesession/subject{subject_id}/trained_model"
    
    # Open models from file
    lowest_loss_model = TF_tools.load_lowest_loss_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    highest_accuracy_model = TF_tools.load_highest_accuracy_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    
    # Get data from files
    with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/testdata-x.pickle", 'rb') as f:
        X_test = pickle.load(f)
    with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/testdata-y.pickle", 'rb') as f:
        y_test = pickle.load(f)
        
    # Get OHE from file
    with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/ohe-encoder.pickle", 'rb') as f:
        ohe = pickle.load(f)
        
    # Get history from file
    with open(f"saved_variables/6/samesubject_samesession/subject{subject_id}/fitting_history.pickle", 'rb') as f:
        history = pickle.load(f)
        
    # Convert OHE labels back to true labels
    y_test = ohe.inverse_transform(y_test)
    
    ################### history stats ###################
    print("#### results of training ####")
    print(f"Best training accuracy (max) {np.round(np.max(history['accuracy']), 4)} @ epoch {np.argmax(history['accuracy']) + 1}")
    print(f"Best training loss (min) {np.round(np.min(history['loss']), 4)} @ epoch {np.argmin(history['loss']) + 1}")
    print()
    print(f"Best validation accuracy (max) {np.round(np.max(history['val_accuracy']), 4)} @ epoch {np.argmax(history['val_accuracy']) + 1}")
    print(f"Best validation loss (min) {np.round(np.min(history['val_loss']), 4)} @ epoch {np.argmin(history['val_loss']) + 1}")
    
    ################### highest accuracy model ###################
    print("\n#### results for highest accuracy model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = highest_accuracy_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### lowest loss model ###################
    print("\n#### results for lowest loss model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = lowest_loss_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### cleanup ###################
    # remove unused vars
    del best_base_model_filename
    del lowest_loss_model
    del highest_accuracy_model
    del f
    del X_test
    del y_test
    del history
    del ohe
    del y_pred
    del accuracy

# Remove unsused variables
del subject_ids_to_test
del subject_id

### Longer window length

An experiment was performed where the window size was changed to 1.5 seconds.
This is done by including 0.25 seconds before and after the queue is shown, totalling 0.25 + 1 + 0.25 = 1.5 seconds.
The following parameters were changed:
- Samples = 300 (from 100)
- dropoutRate = 0.5 (from 0.8)
- conv_filters = 25 (from 10)
- strides = 15 (from 4)
- pool_size = 75 (from 30)

The performed experiment is equal besides this.
The following results were obtained:

| **Subject** | **ShallowConvNet: best validation accuracy** | **ShallowConvNet: best validation loss** | **ShallowConvNet: test split accuracy (best acc model)** | **ShallowConvNet: test split accuracy (best loss model)** |
|-------------|-------------------------------------------|---------------------------------------|-------------------------------------------------------|--------------------------------------------------------|
| B           | 0.8438 @ epoch 177                        | 0.4714 @ epoch 77                     | 0.7552                                                | 0.75                                                   |
| C           | 0.9271 @ epoch 1430                       | 0.3141 @ epoch 2271                   | 0.9531                                                | 0.9688                                                 |
| E           | 0.9686 @ epoch 717                        | 0.1678 @ epoch 232                    | 0.8901                                                | 0.8482                                                 |

The training plots are given below.
B is dark blue, C is light blue and E is green.

![Accuracy plot](figures/6/samesubject_samesession_longer_window/accuracy.png)
![Loss plot](figures/6/samesubject_samesession_longer_window/loss.png)

In [18]:
####################################################
# TRAINING SHALLOWCONVNET FOR EACH SUBJECT AND SESSION
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

do_experiment = True # Long experiment disabled per default

# Function to create the TensorFlow Keras model as clone_model doesn't work with custom objects 
def get_keras_shallowcn_model():
        return ShallowConvNet(
                nb_classes = 3, # int, number of classes to classify. 
                Chans = 21, # number of channels in the EEG data. 
                Samples = 300, # number of time points in the EEG data. (default: 128, paper: 250)
                dropoutRate = 0.5, # dropout fraction. (default: 0.5)
                conv_filters = 25, # Conv2D kernel size (default: 13, paper: 25)
                strides = 15, # Stride size for average pooling layer (default: 7, paper: 15)
                pool_size = 75 # Pool size for average pooling layer (default: 35, paper: 75)
                )

if do_experiment:
        # Loop over all subjects and perform the grid search for finding the best parameters
        for subject_id in subject_ids_to_test:
                print("")
                print("####################################################")
                print(f"# TRAINING FOR SUBJECT {subject_id}")
                print("####################################################")
                print("")
                ###################### PREPARE DATA ######################
                
                with io.capture_output():
                        # Get MNE raw object for latest recording of that subject
                        mne_raw = CLA_dataset.get_last_raw_mne_data_for_subject(subject_id= subject_id)
                        
                        # Get epochs for that MNE raw
                        mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                                             start_offset= start_offset,
                                                                             end_offset= end_offset,
                                                                             baseline= baseline)
                        
                        # Only keep epochs from the MI tasks
                        mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']
                        
                        # Load epochs into memory
                        mne_epochs.load_data()
                
                # Get the labels
                labels = mne_epochs.events[:, -1]
                
                # Convert the labels to OHE labels as needed for Keras
                labels = labels.reshape(-1, 1)
                ohe = OneHotEncoder()
                labels = ohe.fit_transform(labels).toarray()
                
                # Get a 1.5 second window
                mne_epochs_data = mne_epochs.get_data(tmin= -0.25, tmax= 1.25)
                
                # Fix scaling sensitivity as MNE stores as data * 10e-6
                mne_epochs_data = mne_epochs_data * 1000000
                
                # Create a test and train split
                X_train, X_test, y_train, y_test = train_test_split(mne_epochs_data,
                                                                    labels,
                                                                    test_size = 0.2,
                                                                    shuffle= True,
                                                                    stratify= labels,                                                    
                                                                    random_state= 1998)
                
                # Store the train and test data so the best model can be retrained later
                with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/testdata-x.pickle", 'wb') as file:
                        pickle.dump(X_test, file)
                with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/testdata-y.pickle", 'wb') as file:
                        pickle.dump(y_test, file)
                with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/traindata-x.pickle", 'wb') as file:
                        pickle.dump(X_train, file)
                with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/traindata-y.pickle", 'wb') as file:
                        pickle.dump(y_train, file)
                        
                # Store the OHE encoder to enable same conversion later
                with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/ohe-encoder.pickle", 'wb') as file:
                        pickle.dump(ohe, file)
                        
                print(f"Shape of all data (epochs, channels, samples): {np.shape(mne_epochs_data)}")
                print(f"Shape of train data (epochs, channels, samples): {np.shape(X_train)}")
                print(f"Shape of test data (epochs, channels, samples): {np.shape(X_test)}")
                
                
                ###################### PREPARE MODEL ######################
                
                # Names for model
                best_base_model_filename = f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/trained_model" 
                tensorboard_name = f"paper-notebook6_deepconvnet_singlesession_longer_window_subject{subject_id}" # log name for tensorboard
                
                # Create copy of the model that needs training
                trained_model = get_keras_shallowcn_model()
                
                # Compile the model so it can be fitted (loss and optimizer from ShallowConvNet paper)
                trained_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
                
                # Train model with GPU
                # NOTE: change GPU to CPU if nog GPU present
                with tf.device('/gpu:0'):
                        history = trained_model.fit(
                                x= X_train,
                                y= y_train,
                                batch_size= 128, # Default: 32
                                epochs= 2500, # Default: 500
                                verbose= 1, # 0 = silent, 1 = progress bar, 2 = one line per epoch
                                callbacks= [TF_tools.tensorboard_callback(log_name= tensorboard_name),
                                            TF_tools.lowest_loss_model_save_callback(filepath= best_base_model_filename),
                                            TF_tools.highest_accuracy_model_save_callback(filepath= best_base_model_filename)],
                                validation_split= 0.25,
                                shuffle= True,
                                sample_weight= None, # Can be interesting due to time series
                                use_multiprocessing=True, # Done for faster speed
                                workers= 4 # Done for faster speed
                                )
                        
                # Store the fitting history
                with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/fitting_history.pickle", 'wb') as file:
                        pickle.dump(history.history, file)
                
                # Delete vars after singular experiment
                del mne_raw
                del mne_epochs
                del mne_epochs_data
                del trained_model
                del best_base_model_filename
                del tensorboard_name
                del labels
                del file
                del history
                del X_train
                del X_test
                del ohe
                del y_train
                del y_test
    
        # Delete vars after all experiments
        del subject_id
        
# Del global vars
del subject_ids_to_test
del baseline
del do_experiment
del end_offset
del start_offset


####################################################
# TRAINING FOR SUBJECT B
####################################################

Shape of all data (epochs, channels, samples): (960, 21, 300)
Shape of train data (epochs, channels, samples): (768, 21, 300)
Shape of test data (epochs, channels, samples): (192, 21, 300)
Epoch 1/2500
5/5 [==============================] - ETA: 0s - loss: 1.2988 - accuracy: 0.3420
Epoch 1: val_loss improved from inf to 1.10808, saving model to saved_variables/6/samesubject_samesession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 1: val_accuracy improved from -inf to 0.39062, saving model to saved_variables/6/samesubject_samesession_longer_window/subjectB\trained_model_highest_acc_model.hdf5
5/5 [==============================] - 1s 102ms/step - loss: 1.2988 - accuracy: 0.3420 - val_loss: 1.1081 - val_accuracy: 0.3906
Epoch 2/2500
5/5 [==============================] - ETA: 0s - loss: 1.0416 - accuracy: 0.4323
Epoch 2: val_loss impro

3/5 [=================>............] - ETA: 0s - loss: 0.4761 - accuracy: 0.8385
Epoch 16: val_loss improved from 0.82975 to 0.82869, saving model to saved_variables/6/samesubject_samesession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 16: val_accuracy improved from 0.64583 to 0.67188, saving model to saved_variables/6/samesubject_samesession_longer_window/subjectB\trained_model_highest_acc_model.hdf5
5/5 [==============================] - 0s 59ms/step - loss: 0.4570 - accuracy: 0.8472 - val_loss: 0.8287 - val_accuracy: 0.6719
Epoch 17/2500
3/5 [=================>............] - ETA: 0s - loss: 0.4330 - accuracy: 0.8620
Epoch 17: val_loss did not improve from 0.82869

Epoch 17: val_accuracy did not improve from 0.67188
5/5 [==============================] - 0s 40ms/step - loss: 0.4349 - accuracy: 0.8663 - val_loss: 0.8354 - val_accuracy: 0.6667
Epoch 18/2500
3/5 [=================>............] - ETA: 0s - loss: 0.4186 - accuracy: 0.8594
Epoch 18: val_loss improv

Epoch 37/2500
4/5 [=======================>......] - ETA: 0s - loss: 0.2325 - accuracy: 0.9668
Epoch 37: val_loss improved from 0.75510 to 0.74715, saving model to saved_variables/6/samesubject_samesession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 37: val_accuracy did not improve from 0.71875
5/5 [==============================] - 0s 40ms/step - loss: 0.2321 - accuracy: 0.9670 - val_loss: 0.7471 - val_accuracy: 0.7083
Epoch 38/2500
4/5 [=======================>......] - ETA: 0s - loss: 0.2241 - accuracy: 0.9785
Epoch 38: val_loss did not improve from 0.74715

Epoch 38: val_accuracy did not improve from 0.71875
5/5 [==============================] - 0s 34ms/step - loss: 0.2238 - accuracy: 0.9774 - val_loss: 0.7629 - val_accuracy: 0.7083
Epoch 39/2500
3/5 [=================>............] - ETA: 0s - loss: 0.2277 - accuracy: 0.9661
Epoch 39: val_loss did not improve from 0.74715

Epoch 39: val_accuracy did not improve from 0.71875
5/5 [============================

Epoch 59/2500
3/5 [=================>............] - ETA: 0s - loss: 0.1646 - accuracy: 0.9870
Epoch 59: val_loss did not improve from 0.70317

Epoch 59: val_accuracy did not improve from 0.72917
5/5 [==============================] - 0s 40ms/step - loss: 0.1633 - accuracy: 0.9913 - val_loss: 0.7149 - val_accuracy: 0.7240
Epoch 60/2500
5/5 [==============================] - ETA: 0s - loss: 0.1592 - accuracy: 0.9965
Epoch 60: val_loss did not improve from 0.70317

Epoch 60: val_accuracy did not improve from 0.72917
5/5 [==============================] - 0s 42ms/step - loss: 0.1592 - accuracy: 0.9965 - val_loss: 0.7236 - val_accuracy: 0.6979
Epoch 61/2500
3/5 [=================>............] - ETA: 0s - loss: 0.1522 - accuracy: 0.9922
Epoch 61: val_loss did not improve from 0.70317

Epoch 61: val_accuracy did not improve from 0.72917
5/5 [==============================] - 0s 38ms/step - loss: 0.1658 - accuracy: 0.9896 - val_loss: 0.7139 - val_accuracy: 0.7083
Epoch 62/2500
3/5 [=========


Epoch 82: val_accuracy did not improve from 0.74479
5/5 [==============================] - 0s 41ms/step - loss: 0.1328 - accuracy: 0.9983 - val_loss: 0.6641 - val_accuracy: 0.7240
Epoch 83/2500
3/5 [=================>............] - ETA: 0s - loss: 0.1257 - accuracy: 0.9974
Epoch 83: val_loss did not improve from 0.66408

Epoch 83: val_accuracy did not improve from 0.74479
5/5 [==============================] - 0s 35ms/step - loss: 0.1330 - accuracy: 0.9965 - val_loss: 0.6702 - val_accuracy: 0.7292
Epoch 84/2500
3/5 [=================>............] - ETA: 0s - loss: 0.1364 - accuracy: 0.9974
Epoch 84: val_loss improved from 0.66408 to 0.66325, saving model to saved_variables/6/samesubject_samesession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 84: val_accuracy did not improve from 0.74479
5/5 [==============================] - 0s 42ms/step - loss: 0.1379 - accuracy: 0.9931 - val_loss: 0.6633 - val_accuracy: 0.7448
Epoch 85/2500
3/5 [=================>...........

...

5/5 [==============================] - 0s 34ms/step - loss: 0.0750 - accuracy: 0.9965 - val_loss: 0.6041 - val_accuracy: 0.7656
Epoch 602/2500
4/5 [=======================>......] - ETA: 0s - loss: 0.0749 - accuracy: 1.0000
Epoch 602: val_loss did not improve from 0.58056

Epoch 602: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 34ms/step - loss: 0.0771 - accuracy: 1.0000 - val_loss: 0.6001 - val_accuracy: 0.7708
Epoch 603/2500
3/5 [=================>............] - ETA: 0s - loss: 0.0702 - accuracy: 1.0000
Epoch 603: val_loss did not improve from 0.58056

Epoch 603: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 34ms/step - loss: 0.0757 - accuracy: 0.9983 - val_loss: 0.5988 - val_accuracy: 0.7708
Epoch 604/2500
3/5 [=================>............] - ETA: 0s - loss: 0.0811 - accuracy: 0.9974
Epoch 604: val_loss did not improve from 0.58056

Epoch 604: val_accuracy did not improve from 0.80208
5/5 [==============

Epoch 627/2500
3/5 [=================>............] - ETA: 0s - loss: 0.0762 - accuracy: 1.0000
Epoch 627: val_loss did not improve from 0.58056

Epoch 627: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 35ms/step - loss: 0.0763 - accuracy: 1.0000 - val_loss: 0.6060 - val_accuracy: 0.7708
Epoch 628/2500
3/5 [=================>............] - ETA: 0s - loss: 0.0846 - accuracy: 0.9974
Epoch 628: val_loss did not improve from 0.58056

Epoch 628: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 34ms/step - loss: 0.0809 - accuracy: 0.9983 - val_loss: 0.5873 - val_accuracy: 0.7812
Epoch 629/2500
3/5 [=================>............] - ETA: 0s - loss: 0.0699 - accuracy: 1.0000
Epoch 629: val_loss did not improve from 0.58056

Epoch 629: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 35ms/step - loss: 0.0728 - accuracy: 1.0000 - val_loss: 0.5934 - val_accuracy: 0.7708
Epoch 630/2500
3/5 

3/5 [=================>............] - ETA: 0s - loss: 0.0756 - accuracy: 1.0000
Epoch 652: val_loss did not improve from 0.58056

Epoch 652: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 34ms/step - loss: 0.0741 - accuracy: 1.0000 - val_loss: 0.5931 - val_accuracy: 0.7812
Epoch 653/2500
3/5 [=================>............] - ETA: 0s - loss: 0.0748 - accuracy: 1.0000
Epoch 653: val_loss did not improve from 0.58056

Epoch 653: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 34ms/step - loss: 0.0760 - accuracy: 1.0000 - val_loss: 0.6040 - val_accuracy: 0.7760
Epoch 654/2500
3/5 [=================>............] - ETA: 0s - loss: 0.0701 - accuracy: 1.0000
Epoch 654: val_loss did not improve from 0.58056

Epoch 654: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 34ms/step - loss: 0.0729 - accuracy: 0.9983 - val_loss: 0.6135 - val_accuracy: 0.7760
Epoch 655/2500
4/5 [==============

5/5 [==============================] - ETA: 0s - loss: 0.0798 - accuracy: 0.9983
Epoch 677: val_loss did not improve from 0.58056

Epoch 677: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 43ms/step - loss: 0.0798 - accuracy: 0.9983 - val_loss: 0.5956 - val_accuracy: 0.7604
Epoch 678/2500
3/5 [=================>............] - ETA: 0s - loss: 0.0768 - accuracy: 0.9974
Epoch 678: val_loss did not improve from 0.58056

Epoch 678: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 39ms/step - loss: 0.0780 - accuracy: 0.9983 - val_loss: 0.6186 - val_accuracy: 0.7604
Epoch 679/2500
3/5 [=================>............] - ETA: 0s - loss: 0.0780 - accuracy: 1.0000
Epoch 679: val_loss did not improve from 0.58056

Epoch 679: val_accuracy did not improve from 0.80208
5/5 [==============================] - 0s 36ms/step - loss: 0.0772 - accuracy: 1.0000 - val_loss: 0.6082 - val_accuracy: 0.7656
Epoch 680/2500
3/5 [==============

KeyboardInterrupt: 

In [ ]:
####################################################
# RESULTS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings

# Loop over all found results
for subject_id in subject_ids_to_test:
    print()
    print("####################################################")
    print(f"# RESULTS FOR SUBJECT {subject_id}")
    print("####################################################")
    print()
    
    ################### load data ###################
    # Names for model
    best_base_model_filename = f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/trained_model"
    
    # Open models from file
    lowest_loss_model = TF_tools.load_lowest_loss_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    highest_accuracy_model = TF_tools.load_highest_accuracy_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    
    # Get data from files
    with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/testdata-x.pickle", 'rb') as f:
        X_test = pickle.load(f)
    with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/testdata-y.pickle", 'rb') as f:
        y_test = pickle.load(f)
        
    # Get OHE from file
    with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/ohe-encoder.pickle", 'rb') as f:
        ohe = pickle.load(f)
        
    # Get history from file
    with open(f"saved_variables/6/samesubject_samesession_longer_window/subject{subject_id}/fitting_history.pickle", 'rb') as f:
        history = pickle.load(f)
        
    # Convert OHE labels back to true labels
    y_test = ohe.inverse_transform(y_test)
    
    ################### history stats ###################
    print("#### results of training ####")
    print(f"Best training accuracy (max) {np.round(np.max(history['accuracy']), 4)} @ epoch {np.argmax(history['accuracy']) + 1}")
    print(f"Best training loss (min) {np.round(np.min(history['loss']), 4)} @ epoch {np.argmin(history['loss']) + 1}")
    print()
    print(f"Best validation accuracy (max) {np.round(np.max(history['val_accuracy']), 4)} @ epoch {np.argmax(history['val_accuracy']) + 1}")
    print(f"Best validation loss (min) {np.round(np.min(history['val_loss']), 4)} @ epoch {np.argmin(history['val_loss']) + 1}")
    
    ################### highest accuracy model ###################
    print("\n#### results for highest accuracy model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = highest_accuracy_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### lowest loss model ###################
    print("\n#### results for lowest loss model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = lowest_loss_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### cleanup ###################
    # remove unused vars
    del best_base_model_filename
    del lowest_loss_model
    del highest_accuracy_model
    del f
    del X_test
    del y_test
    del history
    del ohe
    del y_pred
    del accuracy

# Remove unsused variables
del subject_ids_to_test
del subject_id

<hr><hr>

## Same subject, new session

As discussed in the master's thesis, training and testing a classification system can happen using multiple strategies.
A classifier may be trained on a singular subject, but by using one or more sessions for training and testing on a new, unseen session.
This is a harder task than the previous one, where training and testing were done for the same session.
This section will train the same classifiers for the same participants as before but by using the first two datasets as training data and the third and final session of each participant as a standalone test set which is not used in training.

This experiment works as follows:
   - We use participants with at least three recordings
      - Participants: B, C, E
      - NOTE: participant F has three files provided but one of those files has only three MI classes rather than three, hence it is not considered here
   - We use the first two recorded session of each of these participants for training and the last for testing.
      - Thus, the CV scores are on the test split for the training data whilst the independent test set is from the unseen session not used during training. This avoids data leakage.
   - We get epochs of 3 seconds, which includes one second before and after the visual queue
      - We use only a half a second window taking into account the online system will use sliding windows.
      - This window starts at 0.1 seconds after then visual queue and ends at 0.6 seconds after the visual queue
   - We use the parameters for ShallowConvNet as found in the experimental notebook `6-DL-based-classification.ipynb`:
      - We input the raw data, which is not baseline corrected, but multiply it by 1000000 to combat the scaling that MNE does per default.
         - This means that we only use the data from the window, so it is easier adoptable to a live setting.
      - We used the modified ShallowConvNet model provided through the `EEGModels.py` util file with the following settings:
         - nb_classes = 3 (int, number of classes to classify)
         - Chans = 21 (number of channels in the EEG data)
         - Samples = 100 (number of time points in the EEG data - default: 128, paper: 250)
         - dropoutRate = 0.5 (dropout fraction - default: 0.5)
         - conv_filters = 10 (Conv2D kernel size - default: 13, paper: 25)
         - strides = 6 (Stride size for average pooling layer - default: 7, paper: 15)
         - pool_size = 30 (Pool size for average pooling layer - default: 35, paper: 75)
      - We trained for 2500 epochs, saving the best model based on best validation accuracy and validation loss (0.25 validation split - equal size as true test split)
   - We record the validation accuracy and loss over time for monitoring the training and test on the seperate test set



In [12]:
####################################################
# TRAINING SHALLOWCONVNET ON EACH SUBJECT AND TWO SESSIONS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

do_experiment = True # Long experiment disabled per default

# Function to create the TensorFlow Keras model as clone_model doesn't work with custom objects 
def get_keras_shallowcn_model():
        return ShallowConvNet(
                nb_classes = 3, # int, number of classes to classify. 
                Chans = 21, # number of channels in the EEG data. 
                Samples = 100, # number of time points in the EEG data. (default: 128, paper: 250)
                dropoutRate = 0.5, # dropout fraction. (default: 0.5)
                conv_filters = 10, # Conv2D kernel size (default: 13, paper: 25)
                strides = 6, # Stride size for average pooling layer (default: 7, paper: 15)
                pool_size = 30 # Pool size for average pooling layer (default: 35, paper: 75)
                )

if do_experiment:
        # Loop over all subjects and perform the grid search for finding the best parameters
        for subject_id in subject_ids_to_test:
                print("")
                print("####################################################")
                print(f"# TRAINING FOR SUBJECT {subject_id}")
                print("####################################################")
                print("")
                ###################### PREPARE DATA ######################
                
                with io.capture_output():
                        # Get all training data (all but last session of participant)
                        mne_raws= CLA_dataset.get_all_but_last_raw_mne_data_for_subject(subject_id= subject_id)
                        
                        # Combine training data into singular mne raw
                        mne_raw = mne.concatenate_raws(mne_raws)
                        
                        # Delete all raws since concat changes them
                        del mne_raws
                        
                        # Get epochs for that MNE raw
                        mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                                             start_offset= start_offset,
                                                                             end_offset= end_offset,
                                                                             baseline= baseline)
                        
                        # Only keep epochs from the MI tasks
                        mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']
                        
                        # Load epochs into memory
                        mne_epochs.load_data()
                
                # Get the labels
                labels = mne_epochs.events[:, -1]
                
                # Convert the labels to OHE labels as needed for Keras
                labels = labels.reshape(-1, 1)
                ohe = OneHotEncoder()
                labels = ohe.fit_transform(labels).toarray()
                
                # Get a half second window
                mne_epochs_data = mne_epochs.get_data(tmin= 0.1, tmax= 0.6)
                
                # Fix scaling sensitivity as MNE stores as data * 10e-6
                mne_epochs_data = mne_epochs_data * 1000000
                        
                # Store the OHE encoder to enable same conversion later
                with open(f"saved_variables/6/samesubject_differentsession/subject{subject_id}/ohe-encoder.pickle", 'wb') as file:
                        pickle.dump(ohe, file)
                        
                print(f"Shape of all data (epochs, channels, samples): {np.shape(mne_epochs_data)}")
                
                
                ###################### PREPARE MODEL ######################
                
                # Names for model
                best_base_model_filename = f"saved_variables/6/samesubject_differentsession/subject{subject_id}/trained_model" 
                tensorboard_name = f"paper-notebook6_deepconvnet_newsession_subject{subject_id}" # log name for tensorboard
                
                # Create copy of the model that needs training
                trained_model = get_keras_shallowcn_model()
                
                # Compile the model so it can be fitted (loss and optimizer from ShallowConvNet paper)
                trained_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
                
                # Train model with GPU
                # NOTE: change GPU to CPU if nog GPU present
                with tf.device('/gpu:0'):
                        history = trained_model.fit(
                                x= mne_epochs_data,
                                y= labels,
                                batch_size= 128, # Default: 32
                                epochs= 2500, # Default: 500
                                verbose= 1, # 0 = silent, 1 = progress bar, 2 = one line per epoch
                                callbacks= [TF_tools.tensorboard_callback(log_name= tensorboard_name),
                                            TF_tools.lowest_loss_model_save_callback(filepath= best_base_model_filename),
                                            TF_tools.highest_accuracy_model_save_callback(filepath= best_base_model_filename)],
                                validation_split= 0.3,
                                shuffle= True,
                                sample_weight= None, # Can be interesting due to time series
                                use_multiprocessing=True, # Done for faster speed
                                workers= 4 # Done for faster speed
                                )
                        
                # Store the fitting history
                with open(f"saved_variables/6/samesubject_differentsession/subject{subject_id}/fitting_history.pickle", 'wb') as file:
                        pickle.dump(history.history, file)
                
                # Delete vars after singular experiment
                del mne_raw
                del mne_epochs
                del mne_epochs_data
                del trained_model
                del best_base_model_filename
                del tensorboard_name
                del labels
                del file
                del history
                del ohe
    
        # Delete vars after all experiments
        del subject_id
        
# Del global vars
del subject_ids_to_test
del baseline
del do_experiment
del end_offset
del start_offset


####################################################
# TRAINING FOR SUBJECT B
####################################################

Shape of all data (epochs, channels, samples): (1918, 21, 100)
Epoch 1/2500
 9/11 [=======================>......] - ETA: 0s - loss: 1.9963 - accuracy: 0.3299
Epoch 1: val_loss improved from inf to 1.12463, saving model to saved_variables/6/samesubject_differentsession/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 1: val_accuracy improved from -inf to 0.35243, saving model to saved_variables/6/samesubject_differentsession/subjectB\trained_model_highest_acc_model.hdf5
11/11 [==============================] - 1s 40ms/step - loss: 1.9497 - accuracy: 0.3241 - val_loss: 1.1246 - val_accuracy: 0.3524
Epoch 2/2500
 9/11 [=======================>......] - ETA: 0s - loss: 1.5200 - accuracy: 0.3498
Epoch 2: val_loss improved from 1.12463 to 1.12347, saving model to saved_variables/6/samesubject_differentsession/subjectB\trained_model_lowest_loss_model.hdf5



11/11 [==============================] - 0s 17ms/step - loss: 1.0114 - accuracy: 0.4918 - val_loss: 1.0604 - val_accuracy: 0.4080
Epoch 20/2500
10/11 [==========================>...] - ETA: 0s - loss: 1.0156 - accuracy: 0.4766
Epoch 20: val_loss did not improve from 1.05720

Epoch 20: val_accuracy did not improve from 0.43056
11/11 [==============================] - 0s 17ms/step - loss: 1.0131 - accuracy: 0.4784 - val_loss: 1.0618 - val_accuracy: 0.4149
Epoch 21/2500
11/11 [==============================] - ETA: 0s - loss: 0.9793 - accuracy: 0.5253
Epoch 21: val_loss improved from 1.05720 to 1.05297, saving model to saved_variables/6/samesubject_differentsession/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 21: val_accuracy did not improve from 0.43056
11/11 [==============================] - 0s 20ms/step - loss: 0.9793 - accuracy: 0.5253 - val_loss: 1.0530 - val_accuracy: 0.4236
Epoch 22/2500
11/11 [==============================] - ETA: 0s - loss: 0.9849 - accuracy: 0.5022
Epo

Epoch 38/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.8596 - accuracy: 0.6062
Epoch 38: val_loss improved from 1.00561 to 1.00375, saving model to saved_variables/6/samesubject_differentsession/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 38: val_accuracy did not improve from 0.50174
11/11 [==============================] - 0s 20ms/step - loss: 0.8569 - accuracy: 0.6095 - val_loss: 1.0038 - val_accuracy: 0.5000
Epoch 39/2500
11/11 [==============================] - ETA: 0s - loss: 0.8612 - accuracy: 0.6058
Epoch 39: val_loss improved from 1.00375 to 0.99847, saving model to saved_variables/6/samesubject_differentsession/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 39: val_accuracy improved from 0.50174 to 0.50868, saving model to saved_variables/6/samesubject_differentsession/subjectB\trained_model_highest_acc_model.hdf5
11/11 [==============================] - 0s 22ms/step - loss: 0.8612 - accuracy: 0.6058 - val_loss: 0.9985 - val_accuracy: 0.5087
Ep

Epoch 59/2500
11/11 [==============================] - ETA: 0s - loss: 0.6771 - accuracy: 0.7340
Epoch 59: val_loss did not improve from 0.95638

Epoch 59: val_accuracy did not improve from 0.54861
11/11 [==============================] - 0s 17ms/step - loss: 0.6771 - accuracy: 0.7340 - val_loss: 0.9608 - val_accuracy: 0.5208
Epoch 60/2500
11/11 [==============================] - ETA: 0s - loss: 0.6743 - accuracy: 0.7385
Epoch 60: val_loss did not improve from 0.95638

Epoch 60: val_accuracy did not improve from 0.54861
11/11 [==============================] - 0s 17ms/step - loss: 0.6743 - accuracy: 0.7385 - val_loss: 0.9671 - val_accuracy: 0.5191
Epoch 61/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.6690 - accuracy: 0.7406
Epoch 61: val_loss did not improve from 0.95638

Epoch 61: val_accuracy did not improve from 0.54861
11/11 [==============================] - 0s 18ms/step - loss: 0.6699 - accuracy: 0.7407 - val_loss: 0.9616 - val_accuracy: 0.5278
Epoch 62/2500
11

Epoch 81/2500
11/11 [==============================] - ETA: 0s - loss: 0.5621 - accuracy: 0.8055
Epoch 81: val_loss did not improve from 0.92228

Epoch 81: val_accuracy improved from 0.56424 to 0.57118, saving model to saved_variables/6/samesubject_differentsession/subjectB\trained_model_highest_acc_model.hdf5
11/11 [==============================] - 0s 20ms/step - loss: 0.5621 - accuracy: 0.8055 - val_loss: 0.9316 - val_accuracy: 0.5712
Epoch 82/2500
11/11 [==============================] - ETA: 0s - loss: 0.5654 - accuracy: 0.8010
Epoch 82: val_loss did not improve from 0.92228

Epoch 82: val_accuracy did not improve from 0.57118
11/11 [==============================] - 0s 17ms/step - loss: 0.5654 - accuracy: 0.8010 - val_loss: 0.9409 - val_accuracy: 0.5451
Epoch 83/2500
11/11 [==============================] - ETA: 0s - loss: 0.5575 - accuracy: 0.8003
Epoch 83: val_loss did not improve from 0.92228

Epoch 83: val_accuracy did not improve from 0.57118
11/11 [=========================

...

Epoch 418/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.3806 - accuracy: 0.9180
Epoch 418: val_loss did not improve from 0.86214

Epoch 418: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 19ms/step - loss: 0.3850 - accuracy: 0.9143 - val_loss: 0.8708 - val_accuracy: 0.6042
Epoch 419/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.3838 - accuracy: 0.9123
Epoch 419: val_loss did not improve from 0.86214

Epoch 419: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 29ms/step - loss: 0.3859 - accuracy: 0.9091 - val_loss: 0.8704 - val_accuracy: 0.5955
Epoch 420/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.3830 - accuracy: 0.9158
Epoch 420: val_loss did not improve from 0.86214

Epoch 420: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 21ms/step - loss: 0.3832 - accuracy: 0.9165 - val_loss: 0.8926 - val_accuracy: 0.6007
Epoch 4

Epoch 443/2500
11/11 [==============================] - ETA: 0s - loss: 0.3879 - accuracy: 0.9069
Epoch 443: val_loss did not improve from 0.86214

Epoch 443: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 25ms/step - loss: 0.3879 - accuracy: 0.9069 - val_loss: 0.8677 - val_accuracy: 0.6059
Epoch 444/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.3589 - accuracy: 0.9358
Epoch 444: val_loss did not improve from 0.86214

Epoch 444: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 18ms/step - loss: 0.3598 - accuracy: 0.9314 - val_loss: 0.8755 - val_accuracy: 0.6059
Epoch 445/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.3788 - accuracy: 0.9242
Epoch 445: val_loss did not improve from 0.86214

Epoch 445: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 19ms/step - loss: 0.3798 - accuracy: 0.9225 - val_loss: 0.8933 - val_accuracy: 0.5938
Epoch 4

Epoch 468/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.3759 - accuracy: 0.9184
Epoch 468: val_loss did not improve from 0.86214

Epoch 468: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 19ms/step - loss: 0.3789 - accuracy: 0.9158 - val_loss: 0.8823 - val_accuracy: 0.5955
Epoch 469/2500
11/11 [==============================] - ETA: 0s - loss: 0.3958 - accuracy: 0.9001
Epoch 469: val_loss did not improve from 0.86214

Epoch 469: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 17ms/step - loss: 0.3958 - accuracy: 0.9001 - val_loss: 0.8636 - val_accuracy: 0.5920
Epoch 470/2500
11/11 [==============================] - ETA: 0s - loss: 0.3734 - accuracy: 0.9195
Epoch 470: val_loss did not improve from 0.86214

Epoch 470: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 17ms/step - loss: 0.3734 - accuracy: 0.9195 - val_loss: 0.8699 - val_accuracy: 0.5990
Epoch 4

Epoch 493/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.3658 - accuracy: 0.9242
Epoch 493: val_loss did not improve from 0.86214

Epoch 493: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 17ms/step - loss: 0.3669 - accuracy: 0.9232 - val_loss: 0.8739 - val_accuracy: 0.5885
Epoch 494/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.3679 - accuracy: 0.9297
Epoch 494: val_loss did not improve from 0.86214

Epoch 494: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 17ms/step - loss: 0.3671 - accuracy: 0.9300 - val_loss: 0.8719 - val_accuracy: 0.5990
Epoch 495/2500
11/11 [==============================] - ETA: 0s - loss: 0.3598 - accuracy: 0.9210
Epoch 495: val_loss did not improve from 0.86214

Epoch 495: val_accuracy did not improve from 0.62847
11/11 [==============================] - 0s 17ms/step - loss: 0.3598 - accuracy: 0.9210 - val_loss: 0.8715 - val_accuracy: 0.6042
Epoch 4

KeyboardInterrupt: 

#### Results

| **Subject** | **ShallowConvNet: best validation accuracy** | **ShallowConvNet: best validation loss** | **ShallowConvNet: test split accuracy (best acc model)** | **ShallowConvNet: test split accuracy (best loss model)** |
|-------------|-------------------------------------------|---------------------------------------|-------------------------------------------------------|--------------------------------------------------------|
| B           | 0.6753 @ epoch 1220                       | 0.7916 @ epoch 428                    | 0.6219                                                | 0.6458                                                 |
| C           | 0.8663 @ epoch 1266                       | 0.43 @ epoch 54                       | 0.7049                                                | 0.7237                                                 |
| E           | 0.7483 @ epoch 79                         | 0.6066 @ epoch 114                    | 0.6838                                                | 0.7351                                                 |

The training plots are given below.
B is dark blue, C is light blue and E is light green.

![Accuracy plot](figures/6/samesubject_newsession/accuracy.png)
![Loss plot](figures/6/samesubject_newsession/loss.png)

In [ ]:
####################################################
# RESULTS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

# Loop over all found results
for subject_id in subject_ids_to_test:
    print()
    print("####################################################")
    print(f"# RESULTS FOR SUBJECT {subject_id}")
    print("####################################################")
    print()
    
    ################### load data ###################
    # Names for model
    best_base_model_filename = f"saved_variables/6/samesubject_differentsession/subject{subject_id}/trained_model"
    
    # Open models from file
    lowest_loss_model = TF_tools.load_lowest_loss_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    highest_accuracy_model = TF_tools.load_highest_accuracy_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    
    # Get test data session
    with io.capture_output():
        # Get test data
        mne_raw = CLA_dataset.get_last_raw_mne_data_for_subject(subject_id)
        
        # Get epochs for test MNE raw
        mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                             start_offset= start_offset,
                                                             end_offset= end_offset,
                                                             baseline= baseline)
        
        # Only keep epochs from the MI tasks
        mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']

        # Load epochs into memory
        mne_epochs.load_data()
        
        # Get the labels
        y_test = mne_epochs.events[:, -1]
        
        # Get a half second window
        X_test = mne_epochs.get_data(tmin= 0.1, tmax= 0.6)
        
        # Fix scaling sensitivity as MNE stores as data * 10e-6
        X_test = X_test * 1000000
        
        # Delete resedual vars for training data
        del mne_raw
        del mne_epochs
        
    # Get OHE from file
    with open(f"saved_variables/6/samesubject_differentsession/subject{subject_id}/ohe-encoder.pickle", 'rb') as f:
        ohe = pickle.load(f)
        
    # Get history from file
    with open(f"saved_variables/6/samesubject_differentsession/subject{subject_id}/fitting_history.pickle", 'rb') as f:
        history = pickle.load(f)
    
    ################### history stats ###################
    print("#### results of training ####")
    print(f"Best training accuracy (max) {np.round(np.max(history['accuracy']), 4)} @ epoch {np.argmax(history['accuracy']) + 1}")
    print(f"Best training loss (min) {np.round(np.min(history['loss']), 4)} @ epoch {np.argmin(history['loss']) + 1}")
    print()
    print(f"Best validation accuracy (max) {np.round(np.max(history['val_accuracy']), 4)} @ epoch {np.argmax(history['val_accuracy']) + 1}")
    print(f"Best validation loss (min) {np.round(np.min(history['val_loss']), 4)} @ epoch {np.argmin(history['val_loss']) + 1}")
    
    ################### highest accuracy model ###################
    print("\n#### results for highest accuracy model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = highest_accuracy_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### lowest loss model ###################
    print("\n#### results for lowest loss model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = lowest_loss_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### cleanup ###################
    # remove unused vars
    del best_base_model_filename
    del lowest_loss_model
    del highest_accuracy_model
    del f
    del X_test
    del y_test
    del history
    del ohe
    del y_pred
    del accuracy

# Remove unsused variables
del subject_ids_to_test
del subject_id
del baseline
del end_offset
del start_offset

### Longer window length

An experiment was performed where the window size was changed to 1.5 seconds.
This is done by including 0.25 seconds before and after the queue is shown, totalling 0.25 + 1 + 0.25 = 1.5 seconds.
The following parameters were changed:
- Samples = 300 (from 100)
- dropoutRate = 0.4 (from 0.5)
- conv_filters = 25 (from 10)
- strides = 15 (from 4)
- pool_size = 75 (from 30)

The performed experiment is equal besides this.
The following results were obtained:


| **Subject** | **ShallowConvNet: best validation accuracy** | **ShallowConvNet: best validation loss** | **ShallowConvNet: test split accuracy (best acc model)** | **ShallowConvNet: test split accuracy (best loss model)** |
|-------------|-------------------------------------------|---------------------------------------|-------------------------------------------------------|--------------------------------------------------------|
| B           | 0.7569 @ epoch 2496                       | 0.7352 @ epoch 95                     | 0.6792                                                | 0.6188                                                 |
| C           | 0.9184 @ epoch 1316                       | 0.4236 @ epoch 406                    | 0.7477                                                | 0.756                                                  |
| E           | 0.8003 @ epoch 54                         | 0.543 @ epoch 39                      | 0.7246                                                | 0.7403                                                 |

The training plots are given below.
B is dark blue, C is light blue and E is green.

![Accuracy plot](figures/6/samesubject_newsession_longer_window/accuracy.png)
![Loss plot](figures/6/samesubject_newsession_longer_window/loss.png)

In [20]:
####################################################
# TRAINING SHALLOWCONVNET ON EACH SUBJECT AND TWO SESSIONS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

do_experiment = True # Long experiment disabled per default

# Function to create the TensorFlow Keras model as clone_model doesn't work with custom objects 
def get_keras_shallowcn_model():
        return ShallowConvNet(
                nb_classes = 3, # int, number of classes to classify. 
                Chans = 21, # number of channels in the EEG data. 
                Samples = 300, # number of time points in the EEG data. (default: 128, paper: 250)
                dropoutRate = 0.4, # dropout fraction. (default: 0.5)
                conv_filters = 25, # Conv2D kernel size (default: 13, paper: 25)
                strides = 15, # Stride size for average pooling layer (default: 7, paper: 15)
                pool_size = 75 # Pool size for average pooling layer (default: 35, paper: 75)
                )

if do_experiment:
        # Loop over all subjects and perform the grid search for finding the best parameters
        for subject_id in subject_ids_to_test:
                print("")
                print("####################################################")
                print(f"# TRAINING FOR SUBJECT {subject_id}")
                print("####################################################")
                print("")
                ###################### PREPARE DATA ######################
                
                with io.capture_output():
                        # Get all training data (all but last session of participant)
                        mne_raws= CLA_dataset.get_all_but_last_raw_mne_data_for_subject(subject_id= subject_id)
                        
                        # Combine training data into singular mne raw
                        mne_raw = mne.concatenate_raws(mne_raws)
                        
                        # Delete all raws since concat changes them
                        del mne_raws
                        
                        # Get epochs for that MNE raw
                        mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                                             start_offset= start_offset,
                                                                             end_offset= end_offset,
                                                                             baseline= baseline)
                        
                        # Only keep epochs from the MI tasks
                        mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']
                        
                        # Load epochs into memory
                        mne_epochs.load_data()
                
                # Get the labels
                labels = mne_epochs.events[:, -1]
                
                # Convert the labels to OHE labels as needed for Keras
                labels = labels.reshape(-1, 1)
                ohe = OneHotEncoder()
                labels = ohe.fit_transform(labels).toarray()
                
                # Get a 1.5 seconds window
                mne_epochs_data = mne_epochs.get_data(tmin= -0.25, tmax= 1.25)
                
                # Fix scaling sensitivity as MNE stores as data * 10e-6
                mne_epochs_data = mne_epochs_data * 1000000
                        
                # Store the OHE encoder to enable same conversion later
                with open(f"saved_variables/6/samesubject_differentsession_longer_window/subject{subject_id}/ohe-encoder.pickle", 'wb') as file:
                        pickle.dump(ohe, file)
                        
                print(f"Shape of all data (epochs, channels, samples): {np.shape(mne_epochs_data)}")
                
                
                ###################### PREPARE MODEL ######################
                
                # Names for model
                best_base_model_filename = f"saved_variables/6/samesubject_differentsession_longer_window/subject{subject_id}/trained_model" 
                tensorboard_name = f"paper-notebook6_deepconvnet_newsession_longer_subject{subject_id}" # log name for tensorboard
                
                # Create copy of the model that needs training
                trained_model = get_keras_shallowcn_model()
                
                # Compile the model so it can be fitted (loss and optimizer from ShallowConvNet paper)
                trained_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
                
                # Train model with GPU
                # NOTE: change GPU to CPU if nog GPU present
                with tf.device('/gpu:0'):
                        history = trained_model.fit(
                                x= mne_epochs_data,
                                y= labels,
                                batch_size= 128, # Default: 32
                                epochs= 2500, # Default: 500
                                verbose= 1, # 0 = silent, 1 = progress bar, 2 = one line per epoch
                                callbacks= [TF_tools.tensorboard_callback(log_name= tensorboard_name),
                                            TF_tools.lowest_loss_model_save_callback(filepath= best_base_model_filename),
                                            TF_tools.highest_accuracy_model_save_callback(filepath= best_base_model_filename)],
                                validation_split= 0.3,
                                shuffle= True,
                                sample_weight= None, # Can be interesting due to time series
                                use_multiprocessing=True, # Done for faster speed
                                workers= 4 # Done for faster speed
                                )
                        
                # Store the fitting history
                with open(f"saved_variables/6/samesubject_differentsession_longer_window/subject{subject_id}/fitting_history.pickle", 'wb') as file:
                        pickle.dump(history.history, file)
                
                # Delete vars after singular experiment
                del mne_raw
                del mne_epochs
                del mne_epochs_data
                del trained_model
                del best_base_model_filename
                del tensorboard_name
                del labels
                del file
                del history
                del ohe
    
        # Delete vars after all experiments
        del subject_id
        
# Del global vars
del subject_ids_to_test
del baseline
del do_experiment
del end_offset
del start_offset


####################################################
# TRAINING FOR SUBJECT B
####################################################

Shape of all data (epochs, channels, samples): (1918, 21, 300)
Epoch 1/2500
 9/11 [=======================>......] - ETA: 0s - loss: 1.8170 - accuracy: 0.3359
Epoch 1: val_loss improved from inf to 1.41999, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 1: val_accuracy improved from -inf to 0.34549, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_highest_acc_model.hdf5
11/11 [==============================] - 1s 68ms/step - loss: 1.7677 - accuracy: 0.3338 - val_loss: 1.4200 - val_accuracy: 0.3455
Epoch 2/2500
 9/11 [=======================>......] - ETA: 0s - loss: 1.4653 - accuracy: 0.3420
Epoch 2: val_loss improved from 1.41999 to 1.30426, saving model to saved_variables/6/samesubject_differentsession_longer_window/subje

10/11 [==========================>...] - ETA: 0s - loss: 0.9836 - accuracy: 0.5266
Epoch 16: val_loss improved from 1.03501 to 1.02510, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 16: val_accuracy did not improve from 0.48438
11/11 [==============================] - 0s 35ms/step - loss: 0.9813 - accuracy: 0.5306 - val_loss: 1.0251 - val_accuracy: 0.4809
Epoch 17/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.9648 - accuracy: 0.5408
Epoch 17: val_loss improved from 1.02510 to 1.01079, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 17: val_accuracy improved from 0.48438 to 0.51215, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_highest_acc_model.hdf5
11/11 [==============================] - 0s 38ms/step - loss: 0.9668 - accuracy: 0.5358 - val_loss: 1.01

Epoch 32/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.7703 - accuracy: 0.6797
Epoch 32: val_loss improved from 0.86974 to 0.85224, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 32: val_accuracy improved from 0.60069 to 0.63542, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_highest_acc_model.hdf5
11/11 [==============================] - 0s 38ms/step - loss: 0.7684 - accuracy: 0.6848 - val_loss: 0.8522 - val_accuracy: 0.6354
Epoch 33/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.7497 - accuracy: 0.7118
Epoch 33: val_loss did not improve from 0.85224

Epoch 33: val_accuracy did not improve from 0.63542
11/11 [==============================] - 0s 32ms/step - loss: 0.7440 - accuracy: 0.7116 - val_loss: 0.8703 - val_accuracy: 0.6059
Epoch 34/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.7081 - accuracy

11/11 [==============================] - 0s 39ms/step - loss: 0.5867 - accuracy: 0.8018 - val_loss: 0.7693 - val_accuracy: 0.6736
Epoch 50/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.5960 - accuracy: 0.7769
Epoch 50: val_loss improved from 0.76552 to 0.76123, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 50: val_accuracy did not improve from 0.68576
11/11 [==============================] - 0s 36ms/step - loss: 0.5887 - accuracy: 0.7854 - val_loss: 0.7612 - val_accuracy: 0.6788
Epoch 51/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.5760 - accuracy: 0.7943
Epoch 51: val_loss improved from 0.76123 to 0.75804, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 51: val_accuracy did not improve from 0.68576
11/11 [==============================] - 0s 38ms/step - loss: 0.5791 - accuracy: 0.7928 

Epoch 70/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.4966 - accuracy: 0.8367
Epoch 70: val_loss improved from 0.72259 to 0.71893, saving model to saved_variables/6/samesubject_differentsession_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 70: val_accuracy did not improve from 0.69618
11/11 [==============================] - 0s 34ms/step - loss: 0.5002 - accuracy: 0.8353 - val_loss: 0.7189 - val_accuracy: 0.6927
Epoch 71/2500
11/11 [==============================] - ETA: 0s - loss: 0.4706 - accuracy: 0.8458
Epoch 71: val_loss did not improve from 0.71893

Epoch 71: val_accuracy did not improve from 0.69618
11/11 [==============================] - 0s 33ms/step - loss: 0.4706 - accuracy: 0.8458 - val_loss: 0.7260 - val_accuracy: 0.6875
Epoch 72/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.4775 - accuracy: 0.8533
Epoch 72: val_loss did not improve from 0.71893

Epoch 72: val_accuracy did not improve from 0.69618
11/11 [===========

...

11/11 [==============================] - 0s 32ms/step - loss: 0.3036 - accuracy: 0.9486 - val_loss: 0.6735 - val_accuracy: 0.7135
Epoch 232/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.3191 - accuracy: 0.9384
Epoch 232: val_loss did not improve from 0.66129

Epoch 232: val_accuracy did not improve from 0.73438
11/11 [==============================] - 0s 32ms/step - loss: 0.3168 - accuracy: 0.9396 - val_loss: 0.6718 - val_accuracy: 0.7188
Epoch 233/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.2978 - accuracy: 0.9531
Epoch 233: val_loss did not improve from 0.66129

Epoch 233: val_accuracy did not improve from 0.73438
11/11 [==============================] - 0s 32ms/step - loss: 0.2982 - accuracy: 0.9545 - val_loss: 0.6619 - val_accuracy: 0.7153
Epoch 234/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.3154 - accuracy: 0.9484
Epoch 234: val_loss did not improve from 0.66129

Epoch 234: val_accuracy did not improve from 0.73438
11/11 [

11/11 [==============================] - 0s 33ms/step - loss: 0.2948 - accuracy: 0.9598 - val_loss: 0.6609 - val_accuracy: 0.7240
Epoch 256/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.3038 - accuracy: 0.9557
Epoch 256: val_loss did not improve from 0.65955

Epoch 256: val_accuracy did not improve from 0.74479
11/11 [==============================] - 0s 34ms/step - loss: 0.3029 - accuracy: 0.9590 - val_loss: 0.6647 - val_accuracy: 0.7188
Epoch 257/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.3026 - accuracy: 0.9547
Epoch 257: val_loss did not improve from 0.65955

Epoch 257: val_accuracy did not improve from 0.74479
11/11 [==============================] - 0s 33ms/step - loss: 0.3011 - accuracy: 0.9553 - val_loss: 0.6615 - val_accuracy: 0.7240
Epoch 258/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.2976 - accuracy: 0.9618
Epoch 258: val_loss did not improve from 0.65955

Epoch 258: val_accuracy did not improve from 0.74479
11/11 [

11/11 [==============================] - 0s 33ms/step - loss: 0.2957 - accuracy: 0.9605 - val_loss: 0.6622 - val_accuracy: 0.7309
Epoch 280/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.2996 - accuracy: 0.9583
Epoch 280: val_loss did not improve from 0.65667

Epoch 280: val_accuracy did not improve from 0.74479
11/11 [==============================] - 0s 32ms/step - loss: 0.2976 - accuracy: 0.9598 - val_loss: 0.6587 - val_accuracy: 0.7361
Epoch 281/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.2954 - accuracy: 0.9575
Epoch 281: val_loss did not improve from 0.65667

Epoch 281: val_accuracy did not improve from 0.74479
11/11 [==============================] - 0s 32ms/step - loss: 0.2971 - accuracy: 0.9575 - val_loss: 0.6627 - val_accuracy: 0.7222
Epoch 282/2500
10/11 [==========================>...] - ETA: 0s - loss: 0.2903 - accuracy: 0.9641
Epoch 282: val_loss did not improve from 0.65667

Epoch 282: val_accuracy did not improve from 0.74479
11/11 [

Epoch 304/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.2829 - accuracy: 0.9670
Epoch 304: val_loss did not improve from 0.64763

Epoch 304: val_accuracy did not improve from 0.74479
11/11 [==============================] - 0s 33ms/step - loss: 0.2897 - accuracy: 0.9627 - val_loss: 0.6561 - val_accuracy: 0.7344
Epoch 305/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.2963 - accuracy: 0.9618
Epoch 305: val_loss did not improve from 0.64763

Epoch 305: val_accuracy did not improve from 0.74479
11/11 [==============================] - 0s 33ms/step - loss: 0.2951 - accuracy: 0.9605 - val_loss: 0.6649 - val_accuracy: 0.7292
Epoch 306/2500
 9/11 [=======================>......] - ETA: 0s - loss: 0.2904 - accuracy: 0.9601
Epoch 306: val_loss did not improve from 0.64763

Epoch 306: val_accuracy did not improve from 0.74479
11/11 [==============================] - 0s 33ms/step - loss: 0.2903 - accuracy: 0.9613 - val_loss: 0.6600 - val_accuracy: 0.7413
Epoch 3

KeyboardInterrupt: 

In [ ]:
####################################################
# RESULTS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

# Loop over all found results
for subject_id in subject_ids_to_test:
    print()
    print("####################################################")
    print(f"# RESULTS FOR SUBJECT {subject_id}")
    print("####################################################")
    print()
    
    ################### load data ###################
    # Names for model
    best_base_model_filename = f"saved_variables/6/samesubject_differentsession_longer_window/subject{subject_id}/trained_model"
    
    # Open models from file
    lowest_loss_model = TF_tools.load_lowest_loss_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    highest_accuracy_model = TF_tools.load_highest_accuracy_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    
    # Get test data session
    with io.capture_output():
        # Get test data
        mne_raw = CLA_dataset.get_last_raw_mne_data_for_subject(subject_id)
        
        # Get epochs for test MNE raw
        mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                             start_offset= start_offset,
                                                             end_offset= end_offset,
                                                             baseline= baseline)
        
        # Only keep epochs from the MI tasks
        mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']

        # Load epochs into memory
        mne_epochs.load_data()
        
        # Get the labels
        y_test = mne_epochs.events[:, -1]
        
        # Get a half second window
        X_test = mne_epochs.get_data(tmin= -0.25, tmax= 1.25)
        
        # Fix scaling sensitivity as MNE stores as data * 10e-6
        X_test = X_test * 1000000
        
        # Delete resedual vars for training data
        del mne_raw
        del mne_epochs
        
    # Get OHE from file
    with open(f"saved_variables/6/samesubject_differentsession_longer_window/subject{subject_id}/ohe-encoder.pickle", 'rb') as f:
        ohe = pickle.load(f)
        
    # Get history from file
    with open(f"saved_variables/6/samesubject_differentsession_longer_window/subject{subject_id}/fitting_history.pickle", 'rb') as f:
        history = pickle.load(f)
    
    ################### history stats ###################
    print("#### results of training ####")
    print(f"Best training accuracy (max) {np.round(np.max(history['accuracy']), 4)} @ epoch {np.argmax(history['accuracy']) + 1}")
    print(f"Best training loss (min) {np.round(np.min(history['loss']), 4)} @ epoch {np.argmin(history['loss']) + 1}")
    print()
    print(f"Best validation accuracy (max) {np.round(np.max(history['val_accuracy']), 4)} @ epoch {np.argmax(history['val_accuracy']) + 1}")
    print(f"Best validation loss (min) {np.round(np.min(history['val_loss']), 4)} @ epoch {np.argmin(history['val_loss']) + 1}")
    
    ################### highest accuracy model ###################
    print("\n#### results for highest accuracy model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = highest_accuracy_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### lowest loss model ###################
    print("\n#### results for lowest loss model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = lowest_loss_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### cleanup ###################
    # remove unused vars
    del best_base_model_filename
    del lowest_loss_model
    del highest_accuracy_model
    del f
    del X_test
    del y_test
    del history
    del ohe
    del y_pred
    del accuracy

# Remove unsused variables
del subject_ids_to_test
del subject_id
del baseline
del end_offset
del start_offset

<hr><hr>

## New subject

As discussed in the master's thesis, training and testing a classification system can happen using multiple strategies.
Perhaps the hardest task is training a classifier on data from one or more subjects, but using it to classify data from a completely new user.
This is the hardest task we'll discuss.
This section will train the same classifiers for the same participants as before but by using one participant for testing and the other two for training.

This experiment works as follows:
   - We use participants with at least three recordings
      - Participants: B, C, E
      - NOTE: participant F has three files provided but one of those files has only three MI classes rather than three, hence it is not considered here
   - We use one participant's data for testing and the other two participant's data for training
      - Thus, the CV scores are on the test split for the training data whilst the independent test set is from the unseen subject not used during training. This avoids data leakage.
   - We get epochs of 3 seconds, which includes one second before and after the visual queue
      - We use only a half a second window taking into account the online system will use sliding windows.
      - This window starts at 0.1 seconds after then visual queue and ends at 0.6 seconds after the visual queue
   - We use the parameters for ShallowConvNet as found in the experimental notebook `6-DL-based-classification.ipynb`:
      - We input the raw data, which is not baseline corrected, but multiply it by 1000000 to combat the scaling that MNE does per default.
         - This means that we only use the data from the window, so it is easier adoptable to a live setting.
      - We used the modified ShallowConvNet model provided through the `EEGModels.py` util file with the following settings:
         - nb_classes = 3 (int, number of classes to classify)
         - Chans = 21 (number of channels in the EEG data)
         - Samples = 100 (number of time points in the EEG data - default: 128, paper: 250)
         - dropoutRate = 0.4 (dropout fraction - default: 0.5)
         - conv_filters = 10 (Conv2D kernel size - default: 13, paper: 25)
         - strides = 6 (Stride size for average pooling layer - default: 7, paper: 15)
         - pool_size = 30 (Pool size for average pooling layer - default: 35, paper: 75)
      - We trained for 2500 epochs, saving the best model based on best validation accuracy and validation loss (0.25 validation split - equal size as true test split)
   - We record the validation accuracy and loss over time for monitoring the training and test on the seperate test set



In [16]:
####################################################
# TRAINING SHALLOWCONVNET ON EACH SUBJECT AND TWO SESSIONS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

do_experiment = True # Long experiment disabled per default

# Function to create the TensorFlow Keras model as clone_model doesn't work with custom objects 
def get_keras_shallowcn_model():
        return ShallowConvNet(
                nb_classes = 3, # int, number of classes to classify. 
                Chans = 21, # number of channels in the EEG data. 
                Samples = 100, # number of time points in the EEG data. (default: 128, paper: 250)
                dropoutRate = 0.4, # dropout fraction. (default: 0.5)
                conv_filters = 10, # Conv2D kernel size (default: 13, paper: 25)
                strides = 6, # Stride size for average pooling layer (default: 7, paper: 15)
                pool_size = 30 # Pool size for average pooling layer (default: 35, paper: 75)
                )

if do_experiment:
        # Loop over all subjects and perform the grid search for finding the best parameters
        for subject_id in subject_ids_to_test:
                print("")
                print("####################################################")
                print(f"# TRAINING FOR SUBJECT {subject_id}")
                print("####################################################")
                print("")
                ###################### PREPARE DATA ######################
                
                with io.capture_output():
                    # Determine the train subjects
                    train_subjects = copy.deepcopy(subject_ids_to_test)
                    train_subjects.remove(subject_id)
                    
                    mne_raws = []
                    
                    # Get all training data
                    for train_subject in train_subjects:
                        mne_raws.extend(CLA_dataset.get_all_raw_mne_data_for_subject(subject_id= train_subject))
                    
                    # Combine training data into singular mne raw
                    mne_raw = mne.concatenate_raws(mne_raws)
                    
                    # Delete all raws since concat changes them
                    del mne_raws
                    
                    # Get epochs for that MNE raw
                    mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                                         start_offset= start_offset,
                                                                         end_offset= end_offset,
                                                                         baseline= baseline)
                    
                    # Only keep epochs from the MI tasks
                    mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']
                    
                    # Load epochs into memory
                    mne_epochs.load_data()
                    
                # Show training data
                print(f"Using data from participants {train_subjects} to train for testing on participant {subject_id}")
                
                # Get the labels
                labels = mne_epochs.events[:, -1]
                
                # Convert the labels to OHE labels as needed for Keras
                labels = labels.reshape(-1, 1)
                ohe = OneHotEncoder()
                labels = ohe.fit_transform(labels).toarray()
                
                # Get a half second window
                mne_epochs_data = mne_epochs.get_data(tmin= 0.1, tmax= 0.6)
                
                # Fix scaling sensitivity as MNE stores as data * 10e-6
                mne_epochs_data = mne_epochs_data * 1000000
                        
                # Store the OHE encoder to enable same conversion later
                with open(f"saved_variables/6/newsubject/subject{subject_id}/ohe-encoder.pickle", 'wb') as file:
                        pickle.dump(ohe, file)
                        
                print(f"Shape of all training data (epochs, channels, samples): {np.shape(mne_epochs_data)}")
                
                
                ###################### PREPARE MODEL ######################
                
                # Names for model
                best_base_model_filename = f"saved_variables/6/newsubject/subject{subject_id}/trained_model" 
                tensorboard_name = f"paper-notebook6_deepconvnet_newsubject_subject{subject_id}" # log name for tensorboard
                
                # Create copy of the model that needs training
                trained_model = get_keras_shallowcn_model()
                
                # Compile the model so it can be fitted (loss and optimizer from ShallowConvNet paper)
                trained_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
                
                # Train model with GPU
                # NOTE: change GPU to CPU if nog GPU present
                with tf.device('/gpu:0'):
                        history = trained_model.fit(
                                x= mne_epochs_data,
                                y= labels,
                                batch_size= 128, # Default: 32
                                epochs= 2500, # Default: 500
                                verbose= 1, # 0 = silent, 1 = progress bar, 2 = one line per epoch
                                callbacks= [TF_tools.tensorboard_callback(log_name= tensorboard_name),
                                            TF_tools.lowest_loss_model_save_callback(filepath= best_base_model_filename),
                                            TF_tools.highest_accuracy_model_save_callback(filepath= best_base_model_filename)],
                                validation_split= 0.3,
                                shuffle= True,
                                sample_weight= None, # Can be interesting due to time series
                                use_multiprocessing=True, # Done for faster speed
                                workers= 4 # Done for faster speed
                                )
                        
                # Store the fitting history
                with open(f"saved_variables/6/newsubject/subject{subject_id}/fitting_history.pickle", 'wb') as file:
                        pickle.dump(history.history, file)
                
                # Delete vars after singular experiment
                del mne_raw
                del mne_epochs
                del mne_epochs_data
                del train_subjects
                del train_subject
                del trained_model
                del best_base_model_filename
                del tensorboard_name
                del labels
                del file
                del history
                del ohe
    
        # Delete vars after all experiments
        del subject_id
        
# Del global vars
del subject_ids_to_test
del baseline
del do_experiment
del end_offset
del start_offset


####################################################
# TRAINING FOR SUBJECT B
####################################################

Using data from participants ['C', 'E'] to train for testing on participant B
Shape of all training data (epochs, channels, samples): (5751, 21, 100)
Epoch 1/2500
29/32 [==========================>...] - ETA: 0s - loss: 1.0705 - accuracy: 0.4949
Epoch 1: val_loss improved from inf to 0.94194, saving model to saved_variables/6/newsubject/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 1: val_accuracy improved from -inf to 0.55736, saving model to saved_variables/6/newsubject/subjectB\trained_model_highest_acc_model.hdf5
32/32 [==============================] - 2s 28ms/step - loss: 1.0526 - accuracy: 0.5051 - val_loss: 0.9419 - val_accuracy: 0.5574
Epoch 2/2500
29/32 [==========================>...] - ETA: 0s - loss: 0.7650 - accuracy: 0.6727
Epoch 2: val_loss improved from 0.94194 to 0.91468, saving model to saved_variables/6/newsubject/subjectB\train

32/32 [==============================] - 0s 15ms/step - loss: 0.3092 - accuracy: 0.8937 - val_loss: 0.7041 - val_accuracy: 0.6952
Epoch 21/2500
31/32 [============================>.] - ETA: 0s - loss: 0.3046 - accuracy: 0.8896
Epoch 21: val_loss did not improve from 0.65458

Epoch 21: val_accuracy did not improve from 0.73233
32/32 [==============================] - 0s 14ms/step - loss: 0.3039 - accuracy: 0.8902 - val_loss: 0.6759 - val_accuracy: 0.7149
Epoch 22/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2950 - accuracy: 0.8942
Epoch 22: val_loss did not improve from 0.65458

Epoch 22: val_accuracy did not improve from 0.73233
32/32 [==============================] - 0s 14ms/step - loss: 0.2952 - accuracy: 0.8939 - val_loss: 0.6645 - val_accuracy: 0.7225
Epoch 23/2500
31/32 [============================>.] - ETA: 0s - loss: 0.3103 - accuracy: 0.8894
Epoch 23: val_loss did not improve from 0.65458

Epoch 23: val_accuracy did not improve from 0.73233
32/32 [=========

Epoch 45/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2616 - accuracy: 0.9146
Epoch 45: val_loss did not improve from 0.63224

Epoch 45: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 14ms/step - loss: 0.2639 - accuracy: 0.9133 - val_loss: 0.7055 - val_accuracy: 0.7034
Epoch 46/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2593 - accuracy: 0.9148
Epoch 46: val_loss did not improve from 0.63224

Epoch 46: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 14ms/step - loss: 0.2590 - accuracy: 0.9145 - val_loss: 0.7417 - val_accuracy: 0.6756
Epoch 47/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2614 - accuracy: 0.9143
Epoch 47: val_loss did not improve from 0.63224

Epoch 47: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 14ms/step - loss: 0.2609 - accuracy: 0.9143 - val_loss: 0.7059 - val_accuracy: 0.6924
Epoch 48/2500
31

Epoch 70/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2377 - accuracy: 0.9307
Epoch 70: val_loss did not improve from 0.63224

Epoch 70: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 14ms/step - loss: 0.2375 - accuracy: 0.9307 - val_loss: 0.6433 - val_accuracy: 0.7329
Epoch 71/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2299 - accuracy: 0.9317
Epoch 71: val_loss did not improve from 0.63224

Epoch 71: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 14ms/step - loss: 0.2305 - accuracy: 0.9309 - val_loss: 0.6609 - val_accuracy: 0.7254
Epoch 72/2500
30/32 [===========================>..] - ETA: 0s - loss: 0.2389 - accuracy: 0.9234
Epoch 72: val_loss did not improve from 0.63224

Epoch 72: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 15ms/step - loss: 0.2401 - accuracy: 0.9232 - val_loss: 0.6772 - val_accuracy: 0.7126
Epoch 73/2500
30

Epoch 95/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2378 - accuracy: 0.9269
Epoch 95: val_loss did not improve from 0.63024

Epoch 95: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 14ms/step - loss: 0.2364 - accuracy: 0.9277 - val_loss: 0.6889 - val_accuracy: 0.7173
Epoch 96/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2282 - accuracy: 0.9370
Epoch 96: val_loss did not improve from 0.63024

Epoch 96: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 14ms/step - loss: 0.2286 - accuracy: 0.9361 - val_loss: 0.6865 - val_accuracy: 0.7121
Epoch 97/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2272 - accuracy: 0.9309
Epoch 97: val_loss did not improve from 0.63024

Epoch 97: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 14ms/step - loss: 0.2259 - accuracy: 0.9317 - val_loss: 0.6441 - val_accuracy: 0.7352
Epoch 98/2500
29

Epoch 120/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2284 - accuracy: 0.9337
Epoch 120: val_loss did not improve from 0.63024

Epoch 120: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 15ms/step - loss: 0.2285 - accuracy: 0.9339 - val_loss: 0.6887 - val_accuracy: 0.7068
Epoch 121/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2204 - accuracy: 0.9327
Epoch 121: val_loss improved from 0.63024 to 0.62627, saving model to saved_variables/6/newsubject/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 121: val_accuracy did not improve from 0.74450
32/32 [==============================] - 0s 15ms/step - loss: 0.2201 - accuracy: 0.9329 - val_loss: 0.6263 - val_accuracy: 0.7422
Epoch 122/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2248 - accuracy: 0.9335
Epoch 122: val_loss did not improve from 0.62627

Epoch 122: val_accuracy did not improve from 0.74450
32/32 [==============================] - 

Epoch 144/2500
30/32 [===========================>..] - ETA: 0s - loss: 0.2228 - accuracy: 0.9357
Epoch 144: val_loss did not improve from 0.62595

Epoch 144: val_accuracy did not improve from 0.74565
32/32 [==============================] - 1s 16ms/step - loss: 0.2248 - accuracy: 0.9352 - val_loss: 0.6490 - val_accuracy: 0.7346
Epoch 145/2500
31/32 [============================>.] - ETA: 0s - loss: 0.2229 - accuracy: 0.9360
Epoch 145: val_loss did not improve from 0.62595

Epoch 145: val_accuracy did not improve from 0.74565
32/32 [==============================] - 0s 15ms/step - loss: 0.2229 - accuracy: 0.9361 - val_loss: 0.6401 - val_accuracy: 0.7312
Epoch 146/2500
28/32 [=========================>....] - ETA: 0s - loss: 0.2221 - accuracy: 0.9381
Epoch 146: val_loss did not improve from 0.62595

Epoch 146: val_accuracy did not improve from 0.74565
32/32 [==============================] - 1s 16ms/step - loss: 0.2220 - accuracy: 0.9376 - val_loss: 0.7278 - val_accuracy: 0.6784
Epoch 1

KeyboardInterrupt: 

#### Results

| **Subject** | **ShallowConvNet: best validation accuracy** | **ShallowConvNet: best validation loss** | **ShallowConvNet: test split accuracy (best acc model)** | **ShallowConvNet: test split accuracy (best loss model)** |
|-------------|-------------------------------------------|---------------------------------------|-------------------------------------------------------|--------------------------------------------------------|
| B           | 0.7543 @ epoch 1554                       | 0.5906 @ epoch 369                    | 0.6208                                                | 0.625                                                  |
| C           | 0.7132 @ epoch 990                        | 0.6552 @ epoch 1417                   | 0.6121                                                | 0.6194                                                 |
| E           | 0.7656 @ epoch 835                        | 0.5876 @ epoch 406                    | 0.6293                                                | 0.6681                                                 |

The training plots are given below.
B is dark blue, C is light blue and E is light green.

![Accuracy plot](figures/6/newsubject/accuracy.png)
![Loss plot](figures/6/newsubject/loss.png)

In [ ]:
####################################################
# RESULTS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

# Loop over all found results
for subject_id in subject_ids_to_test:
    print()
    print("####################################################")
    print(f"# RESULTS FOR SUBJECT {subject_id}")
    print("####################################################")
    print()
    
    ################### load data ###################
    # Names for model
    best_base_model_filename = f"saved_variables/6/newsubject/subject{subject_id}/trained_model"
    
    # Open models from file
    lowest_loss_model = TF_tools.load_lowest_loss_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    highest_accuracy_model = TF_tools.load_highest_accuracy_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    
    # Get test data session
    with io.capture_output():
        # Get test data
        mne_raw = CLA_dataset.get_last_raw_mne_data_for_subject(subject_id)
        
        # Get epochs for test MNE raw
        mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                             start_offset= start_offset,
                                                             end_offset= end_offset,
                                                             baseline= baseline)
        
        # Only keep epochs from the MI tasks
        mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']

        # Load epochs into memory
        mne_epochs.load_data()
        
        # Get the labels
        y_test = mne_epochs.events[:, -1]
        
        # Get a half second window
        X_test = mne_epochs.get_data(tmin= 0.1, tmax= 0.6)
        
        # Fix scaling sensitivity as MNE stores as data * 10e-6
        X_test = X_test * 1000000
        
        # Delete resedual vars for training data
        del mne_raw
        del mne_epochs
        
    # Get OHE from file
    with open(f"saved_variables/6/newsubject/subject{subject_id}/ohe-encoder.pickle", 'rb') as f:
        ohe = pickle.load(f)
        
    # Get history from file
    with open(f"saved_variables/6/newsubject/subject{subject_id}/fitting_history.pickle", 'rb') as f:
        history = pickle.load(f)
    
    ################### history stats ###################
    print("#### results of training ####")
    print(f"Best training accuracy (max) {np.round(np.max(history['accuracy']), 4)} @ epoch {np.argmax(history['accuracy']) + 1}")
    print(f"Best training loss (min) {np.round(np.min(history['loss']), 4)} @ epoch {np.argmin(history['loss']) + 1}")
    print()
    print(f"Best validation accuracy (max) {np.round(np.max(history['val_accuracy']), 4)} @ epoch {np.argmax(history['val_accuracy']) + 1}")
    print(f"Best validation loss (min) {np.round(np.min(history['val_loss']), 4)} @ epoch {np.argmin(history['val_loss']) + 1}")
    
    ################### highest accuracy model ###################
    print("\n#### results for highest accuracy model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = highest_accuracy_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### lowest loss model ###################
    print("\n#### results for lowest loss model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = lowest_loss_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### cleanup ###################
    # remove unused vars
    del best_base_model_filename
    del lowest_loss_model
    del highest_accuracy_model
    del f
    del X_test
    del y_test
    del history
    del ohe
    del y_pred
    del accuracy

# Remove unsused variables
del subject_ids_to_test
del subject_id
del baseline
del end_offset
del start_offset

### Longer window length

An experiment was performed where the window size was changed to 1.5 seconds.
This is done by including 0.25 seconds before and after the queue is shown, totalling 0.25 + 1 + 0.25 = 1.5 seconds.
The following parameters were changed:
- Samples = 300 (from 100)
- dropoutRate = 0.35 (from 0.4)
- conv_filters = 25 (from 10)
- strides = 15 (from 4)
- pool_size = 75 (from 30)

The performed experiment is equal besides this.
The following results were obtained:


| **Subject** | **ShallowConvNet: best validation accuracy** | **ShallowConvNet: best validation loss** | **ShallowConvNet: test split accuracy (best acc model)** | **ShallowConvNet: test split accuracy (best loss model)** |
|-------------|-------------------------------------------|---------------------------------------|-------------------------------------------------------|--------------------------------------------------------|
| B           | 0.8007 @ epoch 49                         | 0.5126 @ epoch 36                     | 0.6073                                                | 0.5979                                                 |
| C           | 0.7561 @ epoch 271                        | 0.6167 @ epoch 40                     | 0.635                                                 | 0.5829                                                 |
| E           | 0.8293 @ epoch 1172                       | 0.5547 @ epoch 330                    | 0.6901                                                | 0.6555                                                 |


The training plots are given below.
B is dark blue, C is light blue and E is green.


![Accuracy plot](figures/6/samesubject_newsession_longer_window/accuracy.png)
![Loss plot](figures/6/samesubject_newsession_longer_window/loss.png)

In [22]:
####################################################
# TRAINING SHALLOWCONVNET ON EACH SUBJECT AND TWO SESSIONS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

do_experiment = True # Long experiment disabled per default

# Function to create the TensorFlow Keras model as clone_model doesn't work with custom objects 
def get_keras_shallowcn_model():
        return ShallowConvNet(
                nb_classes = 3, # int, number of classes to classify. 
                Chans = 21, # number of channels in the EEG data. 
                Samples = 300, # number of time points in the EEG data. (default: 128, paper: 250)
                dropoutRate = 0.35, # dropout fraction. (default: 0.5)
                conv_filters = 25, # Conv2D kernel size (default: 13, paper: 25)
                strides = 15, # Stride size for average pooling layer (default: 7, paper: 15)
                pool_size = 75 # Pool size for average pooling layer (default: 35, paper: 75)
                )

if do_experiment:
        # Loop over all subjects and perform the grid search for finding the best parameters
        for subject_id in subject_ids_to_test:
                print("")
                print("####################################################")
                print(f"# TRAINING FOR SUBJECT {subject_id}")
                print("####################################################")
                print("")
                ###################### PREPARE DATA ######################
                
                with io.capture_output():
                    # Determine the train subjects
                    train_subjects = copy.deepcopy(subject_ids_to_test)
                    train_subjects.remove(subject_id)
                    
                    mne_raws = []
                    
                    # Get all training data
                    for train_subject in train_subjects:
                        mne_raws.extend(CLA_dataset.get_all_raw_mne_data_for_subject(subject_id= train_subject))
                    
                    # Combine training data into singular mne raw
                    mne_raw = mne.concatenate_raws(mne_raws)
                    
                    # Delete all raws since concat changes them
                    del mne_raws
                    
                    # Get epochs for that MNE raw
                    mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                                         start_offset= start_offset,
                                                                         end_offset= end_offset,
                                                                         baseline= baseline)
                    
                    # Only keep epochs from the MI tasks
                    mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']
                    
                    # Load epochs into memory
                    mne_epochs.load_data()
                    
                # Show training data
                print(f"Using data from participants {train_subjects} to train for testing on participant {subject_id}")
                
                # Get the labels
                labels = mne_epochs.events[:, -1]
                
                # Convert the labels to OHE labels as needed for Keras
                labels = labels.reshape(-1, 1)
                ohe = OneHotEncoder()
                labels = ohe.fit_transform(labels).toarray()
                
                # Get a 1.5 seconds window
                mne_epochs_data = mne_epochs.get_data(tmin= -0.25, tmax= 1.25)
                
                # Fix scaling sensitivity as MNE stores as data * 10e-6
                mne_epochs_data = mne_epochs_data * 1000000
                        
                # Store the OHE encoder to enable same conversion later
                with open(f"saved_variables/6/newsubject_longer_window/subject{subject_id}/ohe-encoder.pickle", 'wb') as file:
                        pickle.dump(ohe, file)
                        
                print(f"Shape of all training data (epochs, channels, samples): {np.shape(mne_epochs_data)}")
                
                
                ###################### PREPARE MODEL ######################
                
                # Names for model
                best_base_model_filename = f"saved_variables/6/newsubject_longer_window/subject{subject_id}/trained_model" 
                tensorboard_name = f"paper-notebook6_deepconvnet_newsubject_long_subject{subject_id}" # log name for tensorboard
                
                # Create copy of the model that needs training
                trained_model = get_keras_shallowcn_model()
                
                # Compile the model so it can be fitted (loss and optimizer from ShallowConvNet paper)
                trained_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
                
                # Train model with GPU
                # NOTE: change GPU to CPU if nog GPU present
                with tf.device('/gpu:0'):
                        history = trained_model.fit(
                                x= mne_epochs_data,
                                y= labels,
                                batch_size= 128, # Default: 32
                                epochs= 2500, # Default: 500
                                verbose= 1, # 0 = silent, 1 = progress bar, 2 = one line per epoch
                                callbacks= [TF_tools.tensorboard_callback(log_name= tensorboard_name),
                                            TF_tools.lowest_loss_model_save_callback(filepath= best_base_model_filename),
                                            TF_tools.highest_accuracy_model_save_callback(filepath= best_base_model_filename)],
                                validation_split= 0.3,
                                shuffle= True,
                                sample_weight= None, # Can be interesting due to time series
                                use_multiprocessing=True, # Done for faster speed
                                workers= 4 # Done for faster speed
                                )
                        
                # Store the fitting history
                with open(f"saved_variables/6/newsubject_longer_window/subject{subject_id}/fitting_history.pickle", 'wb') as file:
                        pickle.dump(history.history, file)
                
                # Delete vars after singular experiment
                del mne_raw
                del mne_epochs
                del mne_epochs_data
                del train_subjects
                del train_subject
                del trained_model
                del best_base_model_filename
                del tensorboard_name
                del labels
                del file
                del history
                del ohe
    
        # Delete vars after all experiments
        del subject_id
        
# Del global vars
del subject_ids_to_test
del baseline
del do_experiment
del end_offset
del start_offset


####################################################
# TRAINING FOR SUBJECT B
####################################################

Using data from participants ['C', 'E'] to train for testing on participant B
Shape of all training data (epochs, channels, samples): (5751, 21, 300)
Epoch 1/2500
30/32 [===========================>..] - ETA: 0s - loss: 1.0128 - accuracy: 0.5115
Epoch 1: val_loss improved from inf to 1.00901, saving model to saved_variables/6/newsubject_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 1: val_accuracy improved from -inf to 0.53360, saving model to saved_variables/6/newsubject_longer_window/subjectB\trained_model_highest_acc_model.hdf5
32/32 [==============================] - 2s 41ms/step - loss: 1.0033 - accuracy: 0.5165 - val_loss: 1.0090 - val_accuracy: 0.5336
Epoch 2/2500
31/32 [============================>.] - ETA: 0s - loss: 0.6819 - accuracy: 0.7256
Epoch 2: val_loss improved from 1.00901 to 0.89290, saving model to saved_variables

32/32 [==============================] - 1s 33ms/step - loss: 0.2371 - accuracy: 0.9319 - val_loss: 0.6207 - val_accuracy: 0.7462
Epoch 19/2500
30/32 [===========================>..] - ETA: 0s - loss: 0.2294 - accuracy: 0.9299
Epoch 19: val_loss did not improve from 0.62072

Epoch 19: val_accuracy did not improve from 0.74623
32/32 [==============================] - 1s 31ms/step - loss: 0.2308 - accuracy: 0.9292 - val_loss: 0.6507 - val_accuracy: 0.7387
Epoch 20/2500
30/32 [===========================>..] - ETA: 0s - loss: 0.2424 - accuracy: 0.9312
Epoch 20: val_loss did not improve from 0.62072

Epoch 20: val_accuracy did not improve from 0.74623
32/32 [==============================] - 1s 31ms/step - loss: 0.2406 - accuracy: 0.9319 - val_loss: 0.6606 - val_accuracy: 0.7370
Epoch 21/2500
30/32 [===========================>..] - ETA: 0s - loss: 0.2315 - accuracy: 0.9299
Epoch 21: val_loss did not improve from 0.62072

Epoch 21: val_accuracy did not improve from 0.74623
32/32 [=========

32/32 [==============================] - 1s 33ms/step - loss: 0.1951 - accuracy: 0.9468 - val_loss: 0.5630 - val_accuracy: 0.7775
Epoch 40/2500
31/32 [============================>.] - ETA: 0s - loss: 0.1908 - accuracy: 0.9514
Epoch 40: val_loss did not improve from 0.56304

Epoch 40: val_accuracy did not improve from 0.77752
32/32 [==============================] - 1s 32ms/step - loss: 0.1916 - accuracy: 0.9511 - val_loss: 0.5837 - val_accuracy: 0.7654
Epoch 41/2500
31/32 [============================>.] - ETA: 0s - loss: 0.1883 - accuracy: 0.9516
Epoch 41: val_loss improved from 0.56304 to 0.56274, saving model to saved_variables/6/newsubject_longer_window/subjectB\trained_model_lowest_loss_model.hdf5

Epoch 41: val_accuracy improved from 0.77752 to 0.77984, saving model to saved_variables/6/newsubject_longer_window/subjectB\trained_model_highest_acc_model.hdf5
32/32 [==============================] - 1s 33ms/step - loss: 0.1895 - accuracy: 0.9511 - val_loss: 0.5627 - val_accuracy: 0

32/32 [==============================] - 1s 34ms/step - loss: 0.1793 - accuracy: 0.9565 - val_loss: 0.5678 - val_accuracy: 0.7723
Epoch 63/2500
30/32 [===========================>..] - ETA: 0s - loss: 0.1844 - accuracy: 0.9549
Epoch 63: val_loss did not improve from 0.54904

Epoch 63: val_accuracy did not improve from 0.78563
32/32 [==============================] - 1s 31ms/step - loss: 0.1835 - accuracy: 0.9560 - val_loss: 0.5776 - val_accuracy: 0.7700
Epoch 64/2500
31/32 [============================>.] - ETA: 0s - loss: 0.1856 - accuracy: 0.9521
Epoch 64: val_loss did not improve from 0.54904

Epoch 64: val_accuracy did not improve from 0.78563
32/32 [==============================] - 1s 33ms/step - loss: 0.1855 - accuracy: 0.9520 - val_loss: 0.5811 - val_accuracy: 0.7706
Epoch 65/2500
31/32 [============================>.] - ETA: 0s - loss: 0.1800 - accuracy: 0.9617
Epoch 65: val_loss did not improve from 0.54904

Epoch 65: val_accuracy did not improve from 0.78563
32/32 [=========

KeyboardInterrupt: 

In [ ]:
####################################################
# RESULTS
####################################################

# Configure global parameters for all experiments
subject_ids_to_test = ["B", "C", "E"] # Subjects with three recordings
start_offset = -1 # One second before visual queue
end_offset = 1 # One second after visual queue
baseline = None # Baseline correction using data before the visual queue

# Loop over all found results
for subject_id in subject_ids_to_test:
    print()
    print("####################################################")
    print(f"# RESULTS FOR SUBJECT {subject_id}")
    print("####################################################")
    print()
    
    ################### load data ###################
    # Names for model
    best_base_model_filename = f"saved_variables/6/newsubject_longer_window/subject{subject_id}/trained_model"
    
    # Open models from file
    lowest_loss_model = TF_tools.load_lowest_loss_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    highest_accuracy_model = TF_tools.load_highest_accuracy_model(filepath= best_base_model_filename, custom_objects= {"square": EEGModels.square, "log": EEGModels.log})
    
    # Get test data session
    with io.capture_output():
        # Get test data
        mne_raw = CLA_dataset.get_last_raw_mne_data_for_subject(subject_id)
        
        # Get epochs for test MNE raw
        mne_epochs = CLA_dataset.get_usefull_epochs_from_raw(mne_raw,
                                                             start_offset= start_offset,
                                                             end_offset= end_offset,
                                                             baseline= baseline)
        
        # Only keep epochs from the MI tasks
        mne_epochs = mne_epochs['task/neutral', 'task/left', 'task/right']

        # Load epochs into memory
        mne_epochs.load_data()
        
        # Get the labels
        y_test = mne_epochs.events[:, -1]
        
        # Get a half second window
        X_test = mne_epochs.get_data(tmin= -0.25, tmax= 1.25)
        
        # Fix scaling sensitivity as MNE stores as data * 10e-6
        X_test = X_test * 1000000
        
        # Delete resedual vars for training data
        del mne_raw
        del mne_epochs
        
    # Get OHE from file
    with open(f"saved_variables/6/newsubject_longer_window/subject{subject_id}/ohe-encoder.pickle", 'rb') as f:
        ohe = pickle.load(f)
        
    # Get history from file
    with open(f"saved_variables/6/newsubject_longer_window/subject{subject_id}/fitting_history.pickle", 'rb') as f:
        history = pickle.load(f)
    
    ################### history stats ###################
    print("#### results of training ####")
    print(f"Best training accuracy (max) {np.round(np.max(history['accuracy']), 4)} @ epoch {np.argmax(history['accuracy']) + 1}")
    print(f"Best training loss (min) {np.round(np.min(history['loss']), 4)} @ epoch {np.argmin(history['loss']) + 1}")
    print()
    print(f"Best validation accuracy (max) {np.round(np.max(history['val_accuracy']), 4)} @ epoch {np.argmax(history['val_accuracy']) + 1}")
    print(f"Best validation loss (min) {np.round(np.min(history['val_loss']), 4)} @ epoch {np.argmin(history['val_loss']) + 1}")
    
    ################### highest accuracy model ###################
    print("\n#### results for highest accuracy model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = highest_accuracy_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### lowest loss model ###################
    print("\n#### results for lowest loss model ####")
    # Get predictions from lowest loss model and convert back to labels
    y_pred = lowest_loss_model.predict(X_test)
    y_pred = ohe.inverse_transform(y_pred)
    
    # Get accuracy score and print it
    accuracy =  accuracy_score(y_test, y_pred)
    print(f"Accuracy of: {np.round(accuracy, 4)}")
    
    # Show CM
    ConfusionMatrixDisplay.from_predictions(y_true= y_test, y_pred= y_pred)
    plt.show()
    
    ################### cleanup ###################
    # remove unused vars
    del best_base_model_filename
    del lowest_loss_model
    del highest_accuracy_model
    del f
    del X_test
    del y_test
    del history
    del ohe
    del y_pred
    del accuracy

# Remove unsused variables
del subject_ids_to_test
del subject_id
del baseline
del end_offset
del start_offset